In [63]:
import xarray as xr
import os
from glob import glob
# import datetime
import re
from datetime import datetime, date, timedelta
from numpy import dtype

In [3]:
app_dir = os.getcwd()
app_dir

'/Users/deepthought/Library/CloudStorage/OneDrive-UniversityofLeeds/EUREC4A/Code/CFizer/dev'

In [4]:
data_dir = os.path.join(os.path.dirname(app_dir), 'test_data')
data_dir

'/Users/deepthought/Library/CloudStorage/OneDrive-UniversityofLeeds/EUREC4A/Code/CFizer/test_data'

In [4]:
os.path.exists(data_dir)

True

In [5]:
target_dir =  os.path.join(os.path.dirname(data_dir), f'{os.path.basename(data_dir)}+processed')
target_dir

'/Users/deepthought/Library/CloudStorage/OneDrive-UniversityofLeeds/EUREC4A/Code/CFizer/test_data+processed'

Target directory ideally won't be a subdirectory of the source directory, because that would preclude easily scanning subdirectories for other NC files.

Load a NetCDF file and extract information from its name. Raise an exception if the file doesn't have an `nc` extension, or if the expected patterns are not found.

In [84]:
filename = 'd20200128_diagnostic_1d_3600.nc'

name_parts = re.split('[_\.]', filename)
print(name_parts)
ext = name_parts.pop()
if ext != 'nc':
    raise ValueError('Not a NetCDF file.')
for part in name_parts:
    print(part)
    if part == 'diagnostic':
        continue
    date_match = re.fullmatch('d[0-9]{8}', part)
    print(date_match)
    if date_match is not None:
        # monc_date_str = date_match.string[1:]
        monc_year = int(date_match.string[1:5])
        monc_month = int(date_match.string[5:7])
        monc_day = int(date_match.string[7:])
        # print(f'{monc_year}-{monc_month}-{monc_day}')
        # print(type(monc_year), type(monc_month), type(monc_day))
        # print(datetime.date(monc_year, monc_month, monc_day))
        monc_date = date(year=monc_year, month=monc_month, day=monc_day)
    else:
        dim_match = re.fullmatch('[0-9]d', part)
        print(dim_match)
        if dim_match is not None:
            n_dims = 1 + int(dim_match.string[0])
        else:
            t_match = re.fullmatch('[0-9]+', part)
            print(t_match)
            if t_match is not None:
                end_time = int(t_match.string)
            else:
                raise ValueError('Unexpected filename format')

print(monc_date, n_dims, end_time)

filepath = os.path.join('../test_data/', filename)
try:
    nc1d_1 = xr.open_dataset(filepath)
except FileNotFoundError:
    print(filepath, 'not found')
    exit(1)
except Exception as e:
    print(e)
    exit(1)


['d20200128', 'diagnostic', '1d', '3600', 'nc']
d20200128
<re.Match object; span=(0, 9), match='d20200128'>
diagnostic
1d
None
<re.Match object; span=(0, 2), match='1d'>
3600
None
None
<re.Match object; span=(0, 4), match='3600'>
2020-01-28 2 3600


In [39]:
nc1d_1

<xarray.Dataset>
Dimensions:                (time_series_300_1800: 2, zn: 121, z: 121, number_options: 664, kvp: 2)
Coordinates:
  * time_series_300_1800   (time_series_300_1800) float64 1.8e+03 3.6e+03
  * zn                     (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
  * z                      (z) float64 0.0 50.0 100.0 ... 5.9e+03 5.95e+03 6e+03
Dimensions without coordinates: number_options, kvp
Data variables: (12/41)
    prefn                  (time_series_300_1800, zn) float32 ...
    rho                    (time_series_300_1800, z) float32 ...
    rhon                   (time_series_300_1800, zn) float32 ...
    thref                  (time_series_300_1800, zn) float32 ...
    thinit                 (time_series_300_1800, zn) float32 ...
    u_wind_mean            (time_series_300_1800, zn) float32 ...
    ...                     ...
    praut_mean             (time_series_300_1800, zn) float32 ...
    pracw_mean             (time_series_300_1800, zn) float32 ...
    prevp_mean             (time_series_300_1800, zn) float32 ...
    psedr_mean             (time_series_300_1800, zn) float32 ...
    psedl_mean             (time_series_300_1800, zn) float32 ...
    options_database       (number_options, kvp) |S150 ...
Attributes:
    title:                         1d
    created:                       17/2/2022 22:55:4
    MONC time:                     3600.0
    MONC timestep:                 2620
    Diagnostic write frequency:    3600.0
    Previous diagnostic write at:  0.0

In [198]:
nc1d_1['time_series_300_1800'].data[-1]

3600.0

In [200]:
time_vars = [d for d in nc1d_1.dims if 'time' in d]
time_vars

['time_series_300_1800']

In [13]:
nc1d_1.copy(deep=True)

<xarray.Dataset>
Dimensions:                (time_series_300_1800: 2, zn: 121, z: 121, number_options: 664, kvp: 2)
Coordinates:
  * time_series_300_1800   (time_series_300_1800) float64 1.8e+03 3.6e+03
  * zn                     (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
  * z                      (z) float64 0.0 50.0 100.0 ... 5.9e+03 5.95e+03 6e+03
Dimensions without coordinates: number_options, kvp
Data variables: (12/41)
    prefn                  (time_series_300_1800, zn) float32 1.019e+05 ... 4...
    rho                    (time_series_300_1800, z) float32 1.18 ... 0.6835
    rhon                   (time_series_300_1800, zn) float32 1.182 ... 0.687
    thref                  (time_series_300_1800, zn) float32 298.7 ... 298.7
    thinit                 (time_series_300_1800, zn) float32 0.0 0.0 ... 0.0
    u_wind_mean            (time_series_300_1800, zn) float32 5.826 ... 6.089
    ...                     ...
    praut_mean             (time_series_300_1800, zn) float32 0.0 0.0 ... 0.0
    pracw_mean             (time_series_300_1800, zn) float32 0.0 0.0 ... 0.0
    prevp_mean             (time_series_300_1800, zn) float32 0.0 0.0 ... 0.0
    psedr_mean             (time_series_300_1800, zn) float32 0.0 0.0 ... 0.0
    psedl_mean             (time_series_300_1800, zn) float32 0.0 0.0 ... 0.0
    options_database       (number_options, kvp) |S150 b'rad_interval' ... b'...
Attributes:
    title:                         1d
    created:                       17/2/2022 22:55:4
    MONC time:                     3600.0
    MONC timestep:                 2620
    Diagnostic write frequency:    3600.0
    Previous diagnostic write at:  0.0

In [14]:
nc1d_1.dims

Frozen({'time_series_300_1800': 2, 'zn': 121, 'z': 121, 'number_options': 664, 'kvp': 2})

In [16]:
nc1d_1['zn'].data + 50

array([  25.,   75.,  125.,  175.,  225.,  275.,  325.,  375.,  425.,
        475.,  525.,  575.,  625.,  675.,  725.,  775.,  825.,  875.,
        925.,  975., 1025., 1075., 1125., 1175., 1225., 1275., 1325.,
       1375., 1425., 1475., 1525., 1575., 1625., 1675., 1725., 1775.,
       1825., 1875., 1925., 1975., 2025., 2075., 2125., 2175., 2225.,
       2275., 2325., 2375., 2425., 2475., 2525., 2575., 2625., 2675.,
       2725., 2775., 2825., 2875., 2925., 2975., 3025., 3075., 3125.,
       3175., 3225., 3275., 3325., 3375., 3425., 3475., 3525., 3575.,
       3625., 3675., 3725., 3775., 3825., 3875., 3925., 3975., 4025.,
       4075., 4125., 4175., 4225., 4275., 4325., 4375., 4425., 4475.,
       4525., 4575., 4625., 4675., 4725., 4775., 4825., 4875., 4925.,
       4975., 5025., 5075., 5125., 5175., 5225., 5275., 5325., 5375.,
       5425., 5475., 5525., 5575., 5625., 5675., 5725., 5775., 5825.,
       5875., 5925., 5975., 6025.])

In [17]:
nc1d_1['zn'].copy(data=nc1d_1['zn'].data + 50)

<xarray.DataArray 'zn' (zn: 121)>
array([  25.,   75.,  125.,  175.,  225.,  275.,  325.,  375.,  425.,  475.,
        525.,  575.,  625.,  675.,  725.,  775.,  825.,  875.,  925.,  975.,
       1025., 1075., 1125., 1175., 1225., 1275., 1325., 1375., 1425., 1475.,
       1525., 1575., 1625., 1675., 1725., 1775., 1825., 1875., 1925., 1975.,
       2025., 2075., 2125., 2175., 2225., 2275., 2325., 2375., 2425., 2475.,
       2525., 2575., 2625., 2675., 2725., 2775., 2825., 2875., 2925., 2975.,
       3025., 3075., 3125., 3175., 3225., 3275., 3325., 3375., 3425., 3475.,
       3525., 3575., 3625., 3675., 3725., 3775., 3825., 3875., 3925., 3975.,
       4025., 4075., 4125., 4175., 4225., 4275., 4325., 4375., 4425., 4475.,
       4525., 4575., 4625., 4675., 4725., 4775., 4825., 4875., 4925., 4975.,
       5025., 5075., 5125., 5175., 5225., 5275., 5325., 5375., 5425., 5475.,
       5525., 5575., 5625., 5675., 5725., 5775., 5825., 5875., 5925., 5975.,
       6025.])
Coordinates:
  * zn       (zn) float64 -25.0 25.0 75.0 ... 5.875e+03 5.925e+03 5.975e+03

In [ ]:
nc1d_1.variables[list(nc1d_1.dims.keys())[0]].attrs

{}

In [ ]:
for k in nc1d_1.dims.keys():
    if k in nc1d_1.variables:
        print(
            k, '-',
            nc1d_1.dims[k], ':',
            nc1d_1.variables[k].data[0], 'to',
            nc1d_1.variables[k].data[-1]
        )

time_series_300_1800 - 2 : 41400.0 to 43200.0
zn - 121 : -25.0 to 5975.0
z - 121 : 0.0 to 6000.0


In [ ]:
for t in nc1d_1.time_series_300_1800.data:
    print(t)

41400.0
43200.0


In [ ]:
for t in nc1d_1['time_series_300_1800'].data:
    print(t)

41400.0
43200.0


In [ ]:
nc1d_1.dims['x'] = 1280

TypeError: 'Frozen' object does not support item assignment

In [ ]:
nc1d_1.coords

Coordinates:
  * time_series_300_1800  (time_series_300_1800) float64 4.14e+04 4.32e+04
  * zn                    (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
  * z                     (z) float64 0.0 50.0 100.0 ... 5.9e+03 5.95e+03 6e+03

In [ ]:
print(os.path.dirname(filepath), os.path.basename(filepath))

../test_data d20200128_diagnostic_1d_43200.nc


`Diagnostic write frequency` corresponds to the time points encoded in the names of the `*diagnostic*.nc` files, while the actual time points contained in the files correspond to the 2nd number in the `time_series` variable name, which is the write frequency.

In [ ]:
float(nc1d_1.attrs['MONC time']) == float(end_time)

True

In [ ]:
nc1d_1.attrs

{'title': '1d',
 'created': '18/2/2022 1:11:23',
 'MONC time': '43200.0',
 'MONC timestep': '49119',
 'Diagnostic write frequency': '3600.0',
 'Previous diagnostic write at': '39600.0'}

In [ ]:
for v in nc1d_1.variables:
    print([a for a in dir(nc1d_1[v]) if a[0] != '_'])

['T', 'all', 'any', 'argmax', 'argmin', 'argsort', 'as_numpy', 'assign_attrs', 'assign_coords', 'astype', 'attrs', 'bfill', 'broadcast_equals', 'broadcast_like', 'chunk', 'chunks', 'chunksizes', 'clip', 'close', 'coarsen', 'combine_first', 'compute', 'conj', 'conjugate', 'coords', 'copy', 'count', 'cumprod', 'cumsum', 'cumulative_integrate', 'curvefit', 'data', 'diff', 'differentiate', 'dims', 'dot', 'drop', 'drop_duplicates', 'drop_isel', 'drop_sel', 'drop_vars', 'dropna', 'dt', 'dtype', 'encoding', 'equals', 'expand_dims', 'ffill', 'fillna', 'from_cdms2', 'from_dict', 'from_iris', 'from_series', 'get_axis_num', 'get_index', 'groupby', 'groupby_bins', 'head', 'identical', 'idxmax', 'idxmin', 'imag', 'indexes', 'integrate', 'interp', 'interp_like', 'interpolate_na', 'isel', 'isin', 'isnull', 'item', 'load', 'loc', 'map_blocks', 'max', 'mean', 'median', 'min', 'name', 'nbytes', 'ndim', 'notnull', 'pad', 'persist', 'pipe', 'plot', 'polyfit', 'prod', 'quantile', 'query', 'rank', 'real', '

In [ ]:
for v in nc1d_1.variables:
    print(f'{v}: {len(nc1d_1[v].dims)}')

time_series_300_1800: 1
prefn: 2
rho: 2
rhon: 2
thref: 2
thinit: 2
u_wind_mean: 2
v_wind_mean: 2
w_wind_mean: 2
uu_mean: 2
vv_mean: 2
ww_mean: 2
www_mean: 2
wwww_mean: 2
theta_mean: 2
vapour_mmr_mean: 2
rh_mean: 2
liquid_mmr_mean: 2
rain_mmr_mean: 2
wqv_ad_mean: 2
wql_ad_mean: 2
wqr_ad_mean: 2
wqv_cn_mean: 2
wql_cn_mean: 2
wqr_cn_mean: 2
uw_mean: 2
vw_mean: 2
uv_mean: 2
wtheta_cn_mean: 2
wtheta_ad_mean: 2
wke_mean: 2
th2_mean: 2
total_cloud_fraction: 2
liquid_cloud_fraction: 2
ice_cloud_fraction: 2
pcond_mean: 2
praut_mean: 2
pracw_mean: 2
prevp_mean: 2
psedr_mean: 2
psedl_mean: 2
zn: 1
z: 1
options_database: 2


In [ ]:
nc1d_1['options_database'].data

array([[b'rad_interval', b'60'],
       [b'subsidence_local_q', b'true'],
       [b'passive_q', b'false'],
       ...,
       [b'right_tank_delta_rh', b'0.0'],
       [b'front_tank_delta_rh', b'0.0'],
       [b'cfl_vt_max', b'1.0']], dtype='|S150')

In [ ]:
nc1d_1['options_database'].data[0][0]

b'rad_interval'

Decode binary string as described [here](https://stackoverflow.com/a/17615424).

In [ ]:
nc1d_1['options_database'].data[0][0].decode('utf-8')

'rad_interval'

In [163]:
options_db = {k.decode('utf-8'): v.decode('utf-8') for [k, v] in nc1d_1['options_database'].data}

In [ ]:
{k:v for k, v in options_db.items() if 'z0' in k}

{'z0th': '0.00020', 'z0': '0.00020'}

In [ ]:
options_db['rad_start_year']

'2020.0'

In [ ]:
float(options_db['rad_start_year'])

2020.0

In [ ]:
float('not a number')

ValueError: could not convert string to float: 'not a number'

In [ ]:
try:
    n = float(options_db['rad_start_year'])
except ValueError:
    # not a number
    n = None
else:
    try:
        n = int(options_db['rad_start_year'])
    except ValueError:
        # not an integer
        pass
print(n)

    

2020.0


In [ ]:
str(bool('true')).lower() == 'true'

True

In [ ]:
str(bool('35.0')).lower() == '35.0'

False

In [ ]:
str(bool('false')).lower() == 'false'

False

In [ ]:
bool('false')

True

In [ ]:
'65.2'.lower()

'65.2'

In [ ]:
len(options_db)

664

In [ ]:
for k, v in options_db.items():
    if v.lower() == 'true':
        options_db[k] = True
    elif v.lower() == 'false':
        options_db[k] = False
    else:
        try:
            float(v)
        except ValueError:
            # not a number
            pass
        else:
            try:
                int(v)
            except ValueError:
                # not an integer
                options_db[k] = float(v)
            else:
                options_db[k] = int(v)
options_db

{'rad_interval': 60,
 'subsidence_local_q': True,
 'passive_q': False,
 'checkpoint_file': 'checkpoint_files/dephy_20200128_dump.nc',
 'galilean_transformation': False,
 'l_pgshd': True,
 'smagorinsky_enabled': True,
 'damping_enabled': True,
 'gammam': 19.3,
 'gammah': 12.0,
 'symm_prob': False,
 'pressure-terms_group_type': 'column',
 'finalisation_stage_orderinga_34': 'conditional_diagnostics_column',
 'finalisation_stage_orderinga_33': 'conditional_diagnostics_whole',
 'finalisation_stage_orderinga_32': 'pstep',
 'finalisation_stage_orderinga_31': 'tracers',
 'finalisation_stage_orderinga_30': 'simplecloud',
 'l_thref': False,
 'finalisation_stage_orderinga_29': 'damping',
 'finalisation_stage_orderinga_28': 'th_advection',
 'finalisation_stage_orderinga_27': 'pw_advection',
 'finalisation_stage_orderinga_26': 'profile_diagnostics',
 'finalisation_stage_orderinga_25': 'viscosity',
 'finalisation_stage_orderinga_24': 'lower_bc',
 'finalisation_stage_orderinga_23': 'diffusion',
 'fin

In [ ]:
len(options_db)

664

In [ ]:
options_db['z0']

0.0002

The above works, but RegEx could do more efficiently.

In [ ]:
{k:v for k, v in options_db.items() if 'rad_start' in k}

{'rad_start_year': 2020.0, 'rad_start_time': 0.0, 'rad_start_day': 25.0}

In [ ]:
odb = {}
for k,v in {k.decode('utf-8'): v.decode('utf-8') for [k, v] in nc1d_1['options_database'].data}.items():
    if re.fullmatch(pattern='[0-9]+\.[0-9]+', string=v):
        odb[k] = float(v)
    elif re.fullmatch(pattern='[0-9]+', string=v):
        odb[k] = int(v)
    elif v.lower() == 'true':
        odb[k] = True
    elif v.lower() == 'false':
        odb[k] = False
    else:
        odb[k] = v

odb

{'rad_interval': 60,
 'subsidence_local_q': True,
 'passive_q': False,
 'checkpoint_file': 'checkpoint_files/dephy_20200128_dump.nc',
 'galilean_transformation': False,
 'l_pgshd': True,
 'smagorinsky_enabled': True,
 'damping_enabled': True,
 'gammam': 19.3,
 'gammah': 12.0,
 'symm_prob': False,
 'pressure-terms_group_type': 'column',
 'finalisation_stage_orderinga_34': 'conditional_diagnostics_column',
 'finalisation_stage_orderinga_33': 'conditional_diagnostics_whole',
 'finalisation_stage_orderinga_32': 'pstep',
 'finalisation_stage_orderinga_31': 'tracers',
 'finalisation_stage_orderinga_30': 'simplecloud',
 'l_thref': False,
 'finalisation_stage_orderinga_29': 'damping',
 'finalisation_stage_orderinga_28': 'th_advection',
 'finalisation_stage_orderinga_27': 'pw_advection',
 'finalisation_stage_orderinga_26': 'profile_diagnostics',
 'finalisation_stage_orderinga_25': 'viscosity',
 'finalisation_stage_orderinga_24': 'lower_bc',
 'finalisation_stage_orderinga_23': 'diffusion',
 'fin

In [ ]:
len(odb)

664

In [ ]:
{k:v for k, v in odb.items() if ("freq" in k or "int" in k)}

{'rad_interval': 60,
 'checkpoint_file': 'checkpoint_files/dephy_20200128_dump.nc',
 'cfl_frequency': 10,
 'force_output_on_interval': False,
 'check_walltime_frequency': 20,
 'diag_write_freq': 3600.0,
 'checkpoint_unique_per_dump': False,
 '3d_sampling_frequency': 1800,
 'print_debug_data': False,
 'display_synopsis_frequency': 100,
 'checkpointer_enabled': True,
 'checkpoint_internal_write': 'auto',
 'checkpoint_frequency': 0,
 'check_msg_frequency': 500,
 'sampling_frequency': 300}

In [ ]:
{k:v for k, v in odb.items() if "monc" in k.lower()}

{'moncs_per_io_server': 15}

In [ ]:
nc1d_1.attrs

{'title': '1d',
 'created': '18/2/2022 1:11:23',
 'MONC time': '43200.0',
 'MONC timestep': '49119',
 'Diagnostic write frequency': '3600.0',
 'Previous diagnostic write at': '39600.0'}

In [ ]:
float(nc1d_1.attrs['Diagnostic write frequency'])


3600.0

In [ ]:
nc1d_1.attrs['sample frequency'] = str(float(odb['sampling_frequency']))

In [ ]:
nc1d_1.attrs

{'title': '1d',
 'created': '18/2/2022 1:11:23',
 'MONC time': '43200.0',
 'MONC timestep': '49119',
 'Diagnostic write frequency': '3600.0',
 'Previous diagnostic write at': '39600.0',
 'sample frequency': '300.0'}

In [ ]:
nc1d_1.attrs.pop('sample frequency')
nc1d_1.attrs


{'title': '1d',
 'created': '18/2/2022 1:11:23',
 'MONC time': '43200.0',
 'MONC timestep': '49119',
 'Diagnostic write frequency': '3600.0',
 'Previous diagnostic write at': '39600.0'}

In [ ]:
[sampling_freq, write_freq] = 'time_series_300_1800'.split('_')[-2:]
sampling_freq

'300'

In [ ]:
for c in nc1d_1.coords:
    if 'time' in c:
        [sampling_freq, write_freq] = [int(n) for n in c.split('_')[-2:]]
        updated_nc = nc1d_1.rename({c: 'time'})
        updated_nc.attrs['Sampling frequency'] = sampling_freq
        updated_nc.attrs['Write frequency'] = write_freq

updated_nc

<xarray.Dataset>
Dimensions:                (time: 2, zn: 121, z: 121, number_options: 664, kvp: 2)
Coordinates:
  * time                   (time) float64 4.14e+04 4.32e+04
  * zn                     (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
  * z                      (z) float64 0.0 50.0 100.0 ... 5.9e+03 5.95e+03 6e+03
Dimensions without coordinates: number_options, kvp
Data variables: (12/41)
    prefn                  (time, zn) float32 1.019e+05 1.013e+05 ... 4.767e+04
    rho                    (time, z) float32 1.18 1.175 1.17 ... 0.6887 0.6835
    rhon                   (time, zn) float32 1.182 1.177 1.172 ... 0.6905 0.687
    thref                  (time, zn) float32 298.7 298.7 298.7 ... 298.7 298.7
    thinit                 (time, zn) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    u_wind_mean            (time, zn) float32 8.031 -8.031 -8.295 ... 4.829 4.84
    ...                     ...
    praut_mean             (time, zn) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    pracw_mean             (time, zn) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    prevp_mean             (time, zn) float32 0.0 2.63e-13 3.767e-13 ... 0.0 0.0
    psedr_mean             (time, zn) float32 0.0 1.183e-12 ... 0.0 0.0
    psedl_mean             (time, zn) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    options_database       (number_options, kvp) |S150 b'rad_interval' ... b'...
Attributes:
    title:                         1d
    created:                       18/2/2022 1:11:23
    MONC time:                     43200.0
    MONC timestep:                 49119
    Diagnostic write frequency:    3600.0
    Previous diagnostic write at:  39600.0
    Sampling frequency:            300
    Write frequency:               1800

In [ ]:
for d in nc1d_1.dims:
    print(d)

nc1d_1.dims


time_series_300_1800
zn
z
number_options
kvp


Frozen({'time_series_300_1800': 2, 'zn': 121, 'z': 121, 'number_options': 664, 'kvp': 2})

In [ ]:
updated_nc.attrs

{'title': '1d',
 'created': '18/2/2022 1:11:23',
 'MONC time': '43200.0',
 'MONC timestep': '49119',
 'Diagnostic write frequency': '3600.0',
 'Previous diagnostic write at': '39600.0',
 'Sampling frequency': 300,
 'Write frequency': 1800}

In [ ]:
print(f"""sampling frequency from time variable matches:
      sampling_frequency: {
          float(updated_nc.attrs['Sampling frequency']) == float(odb['sampling_frequency'])
          }
      3d_sampling_frequency: {
          float(updated_nc.attrs['Sampling frequency']
                ) == float(odb['3d_sampling_frequency'])
      }""")


sampling frequency from time variable matches:
      sampling_frequency: True
      3d_sampling_frequency: False


In [ ]:
print(
    f"Time interval matches write frequency? {float(updated_nc.attrs['Write frequency']) == updated_nc.time.values[1] - updated_nc.time.values[0]}")


Time interval matches write frequency? True


In [ ]:
updated_nc.attrs['Conventions'] = 'CF-1.10'

In [ ]:
updated_nc.attrs

{'title': '1d',
 'created': '18/2/2022 1:11:23',
 'MONC time': '43200.0',
 'MONC timestep': '49119',
 'Diagnostic write frequency': '3600.0',
 'Previous diagnostic write at': '39600.0',
 'Sampling frequency': 300,
 'Write frequency': 1800,
 'Conventions': 'CF-1.10'}

In [ ]:
for k, v in odb.items():
    if 'dephy' in k.lower():
        print(k, v)

dephy_forcings_enabled True
dephy_file eurec4a_20200128_first.dephy_inversion_nudging.nc
dephy_forcing_enabled False


In [ ]:
type(updated_nc)

xarray.core.dataset.Dataset

In [ ]:
type(updated_nc.variables)

xarray.core.utils.Frozen

In [ ]:
updated_nc.rho.units

'kg/m^3'

In [ ]:
updated_nc.rho.attrs

{'units': 'kg/m^3'}

In [ ]:
for variable in updated_nc.variables:
    print(variable, updated_nc[variable].coords, updated_nc[variable].attrs)


time Coordinates:
  * time     (time) float64 4.14e+04 4.32e+04 {}
prefn Coordinates:
  * time     (time) float64 4.14e+04 4.32e+04
  * zn       (zn) float64 -25.0 25.0 75.0 ... 5.875e+03 5.925e+03 5.975e+03 {'units': 'hPa'}
rho Coordinates:
  * time     (time) float64 4.14e+04 4.32e+04
  * z        (z) float64 0.0 50.0 100.0 150.0 ... 5.9e+03 5.95e+03 6e+03 {'units': 'kg/m^3'}
rhon Coordinates:
  * time     (time) float64 4.14e+04 4.32e+04
  * zn       (zn) float64 -25.0 25.0 75.0 ... 5.875e+03 5.925e+03 5.975e+03 {'units': 'kg/m^3'}
thref Coordinates:
  * time     (time) float64 4.14e+04 4.32e+04
  * zn       (zn) float64 -25.0 25.0 75.0 ... 5.875e+03 5.925e+03 5.975e+03 {'units': 'K'}
thinit Coordinates:
  * time     (time) float64 4.14e+04 4.32e+04
  * zn       (zn) float64 -25.0 25.0 75.0 ... 5.875e+03 5.925e+03 5.975e+03 {'units': 'K'}
u_wind_mean Coordinates:
  * time     (time) float64 4.14e+04 4.32e+04
  * zn       (zn) float64 -25.0 25.0 75.0 ... 5.875e+03 5.925e+03 5.975e+03

In [ ]:
updated_nc.rho.coords.dims

('time', 'z')

In [ ]:
updated_nc.rho.attrs.update(standard_name='air_density')
updated_nc.rho.attrs


{'units': 'kg/m^3', 'standard_name': 'air_density'}

In [ ]:
updated_nc.rho.attrs.update(long_name='Density at w levels', units='kg m-3')
updated_nc.rho.attrs

{'units': 'kg m-3',
 'standard_name': 'air_density',
 'long_name': 'Density at w levels'}

In [3]:
import cfunits

rho_units = cfunits.Units('kg/m^3')
rho_units

<Units: kg/m^3>

In [ ]:
hPa = cfunits.Units('hPa')
hPa.equivalent(cfunits.Units('Pa'))

True

In [ ]:
hPa.equivalent(cfunits.Units('N m-2'))

True

In [ ]:
cfunits.Units('kg/kg').isvalid

True

In [ ]:
cfunits.Units('kg/kg').isdimensionless

True

In [ ]:
cfunits.Units('spam').reason_notvalid

"Invalid units: 'spam'; Not recognised by UDUNITS"

In [ ]:
updated_nc.coords

Coordinates:
  * time     (time) float64 4.14e+04 4.32e+04
  * zn       (zn) float64 -25.0 25.0 75.0 ... 5.875e+03 5.925e+03 5.975e+03
  * z        (z) float64 0.0 50.0 100.0 150.0 ... 5.9e+03 5.95e+03 6e+03

In [ ]:
updated_nc.coords['time']

<xarray.DataArray 'time' (time: 2)>
array([41400., 43200.])
Coordinates:
  * time     (time) float64 4.14e+04 4.32e+04

In [ ]:
if cfunits.Units('s').isvalid: updated_nc.coords['time'].attrs['units'] = 's'
updated_nc.coords['time'].attrs['standard_name'] = 'time'
updated_nc.coords['time'].attrs['axis'] = 'T'
updated_nc.coords['time']

<xarray.DataArray 'time' (time: 2)>
array([41400., 43200.])
Coordinates:
  * time     (time) float64 4.14e+04 4.32e+04
Attributes:
    units:          s
    standard_name:  time
    axis:           T

In [ ]:
hPa.formatted()

'100 Pa'

In [ ]:
rho_units.formatted()

'm-3.kg'

In [ ]:
rho_units.units

'kg/m^3'

In [ ]:
hPa.units

'hPa'

In [ ]:
rho_units.equivalent(cfunits.Units('N s^2 / m^4'))

True

In [ ]:
rho_units.equals(cfunits.Units('N s^2 / m^4'))

True

In [ ]:
hPa == cfunits.Units('Pa')

False

In [ ]:
hPa.equivalent(cfunits.Units('Pa'))

True

In [ ]:
rho_units == cfunits.Units('kg m-3')

True

In [ ]:
updated_nc.dims

Frozen({'time': 2, 'zn': 121, 'z': 121, 'number_options': 664, 'kvp': 2})

In [ ]:
updated_nc.coords

Coordinates:
  * time     (time) float64 4.14e+04 4.32e+04
  * zn       (zn) float64 -25.0 25.0 75.0 ... 5.875e+03 5.925e+03 5.975e+03
  * z        (z) float64 0.0 50.0 100.0 150.0 ... 5.9e+03 5.95e+03 6e+03

In [ ]:
updated_nc['z']

<xarray.DataArray 'z' (z: 121)>
array([   0.,   50.,  100.,  150.,  200.,  250.,  300.,  350.,  400.,  450.,
        500.,  550.,  600.,  650.,  700.,  750.,  800.,  850.,  900.,  950.,
       1000., 1050., 1100., 1150., 1200., 1250., 1300., 1350., 1400., 1450.,
       1500., 1550., 1600., 1650., 1700., 1750., 1800., 1850., 1900., 1950.,
       2000., 2050., 2100., 2150., 2200., 2250., 2300., 2350., 2400., 2450.,
       2500., 2550., 2600., 2650., 2700., 2750., 2800., 2850., 2900., 2950.,
       3000., 3050., 3100., 3150., 3200., 3250., 3300., 3350., 3400., 3450.,
       3500., 3550., 3600., 3650., 3700., 3750., 3800., 3850., 3900., 3950.,
       4000., 4050., 4100., 4150., 4200., 4250., 4300., 4350., 4400., 4450.,
       4500., 4550., 4600., 4650., 4700., 4750., 4800., 4850., 4900., 4950.,
       5000., 5050., 5100., 5150., 5200., 5250., 5300., 5350., 5400., 5450.,
       5500., 5550., 5600., 5650., 5700., 5750., 5800., 5850., 5900., 5950.,
       6000.])
Coordinates:
  * z        (z) float64 0.0 50.0 100.0 150.0 ... 5.9e+03 5.95e+03 6e+03

In [ ]:
if cfunits.Units('m').isvalid: updated_nc.z.attrs['units'] = 'm'
updated_nc.z.attrs['axis'] = 'Z'
updated_nc.z.attrs['long_name'] = 'Altitude of vertical velocity levels'
updated_nc.z.attrs

{'units': 'm',
 'axis': 'Z',
 'long_name': 'Altitude of vertical velocity levels'}

In [ ]:
if 'units' in updated_nc.zn.attrs:
    print(f'Current unit: {cfunits.Units(updated_nc.zn.attrs["units"])}')
else:
    if cfunits.Units('m').isvalid: updated_nc.zn.attrs['units'] = 'm'
    print(f'No units specified. New unit added: {updated_nc.zn.attrs["units"]}')

No units specified. New unit added: m


In [ ]:
if 'standard_name' in updated_nc.zn.attrs:
    print(f'Current standard name: {updated_nc.zn.attrs["standard_name"]}')
else:
    updated_nc.zn.attrs['standard_name'] = 'altitude'
    print(
        f'No standard name specified. Added as: {updated_nc.zn.attrs["standard_name"]}')


No standard name specified. Added as: altitude


In [ ]:
updated_nc.zn

<xarray.DataArray 'zn' (zn: 121)>
array([ -25.,   25.,   75.,  125.,  175.,  225.,  275.,  325.,  375.,  425.,
        475.,  525.,  575.,  625.,  675.,  725.,  775.,  825.,  875.,  925.,
        975., 1025., 1075., 1125., 1175., 1225., 1275., 1325., 1375., 1425.,
       1475., 1525., 1575., 1625., 1675., 1725., 1775., 1825., 1875., 1925.,
       1975., 2025., 2075., 2125., 2175., 2225., 2275., 2325., 2375., 2425.,
       2475., 2525., 2575., 2625., 2675., 2725., 2775., 2825., 2875., 2925.,
       2975., 3025., 3075., 3125., 3175., 3225., 3275., 3325., 3375., 3425.,
       3475., 3525., 3575., 3625., 3675., 3725., 3775., 3825., 3875., 3925.,
       3975., 4025., 4075., 4125., 4175., 4225., 4275., 4325., 4375., 4425.,
       4475., 4525., 4575., 4625., 4675., 4725., 4775., 4825., 4875., 4925.,
       4975., 5025., 5075., 5125., 5175., 5225., 5275., 5325., 5375., 5425.,
       5475., 5525., 5575., 5625., 5675., 5725., 5775., 5825., 5875., 5925.,
       5975.])
Coordinates:
  * zn       (zn) float64 -25.0 25.0 75.0 ... 5.875e+03 5.925e+03 5.975e+03
Attributes:
    units:          m
    standard_name:  altitude

In [ ]:
for attribute, std_value in {'standard_name':'altitude', 'long_name':None, 'units':'m', 'axis':'Z'}.items():
    if attribute in updated_nc.zn.attrs:
        print(
            f"{attribute}: {updated_nc.zn.attrs[attribute]}: {['incorrect', 'correct'][int(updated_nc.zn.attrs[attribute] == std_value)]}")
    else:
        if std_value is not None: 
            updated_nc.zn.attrs[attribute] = std_value
            print(
                f"{attribute} not found: set to {updated_nc.zn.attrs[attribute]}"
            )
        else:
            print(
                f"{attribute} not found. No substitution available."
            )


standard_name: altitude: correct
long_name not found. No substitution available.
units: m: correct
axis not found: set to Z


In [ ]:
updated_nc.zn.attrs


{'units': 'm', 'standard_name': 'altitude', 'axis': 'Z'}

In [ ]:
updated_nc.rho

<xarray.DataArray 'rho' (time: 2, z: 121)>
array([[1.179579, 1.174786, 1.170005, ..., 0.692211, 0.688732, 0.68354 ],
       [1.179579, 1.174786, 1.170005, ..., 0.692211, 0.688732, 0.68354 ]],
      dtype=float32)
Coordinates:
  * time     (time) float64 4.14e+04 4.32e+04
  * z        (z) float64 0.0 50.0 100.0 150.0 ... 5.9e+03 5.95e+03 6e+03
Attributes:
    units:          kg m-3
    standard_name:  air_density
    long_name:      Density at w levels

In [ ]:
updated_nc.rhon

<xarray.DataArray 'rhon' (time: 2, zn: 121)>
array([[1.181981, 1.177182, 1.172395, ..., 0.693956, 0.690471, 0.686997],
       [1.181981, 1.177182, 1.172395, ..., 0.693956, 0.690471, 0.686997]],
      dtype=float32)
Coordinates:
  * time     (time) float64 4.14e+04 4.32e+04
  * zn       (zn) float64 -25.0 25.0 75.0 ... 5.875e+03 5.925e+03 5.975e+03
Attributes:
    units:    kg/m^3

In [ ]:
cfunits.Units(updated_nc.rhon.units).formatted()

'm-3.kg'

In [ ]:
nc1d_1.close()
updated_nc.close()

# In production version, use with xr.open_dataset(filepath) as ds: ...
# to avoid need to close afterwards.

In [ ]:
filename = 'CEDA_archived_output.nc'
ceda = xr.open_dataset(os.path.join('../test_data/', filename))


In [ ]:
ceda

<xarray.Dataset>
Dimensions:                  (time: 24, depth: 4, latitude: 2100, longitude: 2000, bnds: 2)
Coordinates:
  * time                     (time) object 2006-07-23 01:00:00 ... 2006-07-24...
  * depth                    (depth) float32 0.05 0.225 0.675 2.0
  * latitude                 (latitude) float32 -45.5 -45.46 ... 39.46 39.5
  * longitude                (longitude) float32 335.5 335.6 ... 416.4 416.5
    forecast_period          (time) timedelta64[ns] ...
    forecast_reference_time  object ...
Dimensions without coordinates: bnds
Data variables:
    e08223                   (time, depth, latitude, longitude) float32 ...
    latitude_longitude       int32 ...
    depth_bnds               (depth, bnds) float32 ...
Attributes: (12/16)
    PP_LBPROC:       0
    PP_LBTIM:        12
    PP_MEAN_PERIOD:  0.0
    contact:         rachel.stratton@metoffice.gov.uk
    copyright:       (C) Crown Copyright, 2015 the Met Office
    creation_date:   2018-02-25T12:12:26Z
    ...              ...
    institution_id:  MOHC
    model:           Regional GA7 configuration on Africa 4.5 km
    model_id:        inst
    project_id:      IMPALA
    source:          Data from Met Office Unified Model
    Conventions:     CF-1.5

In [ ]:
ceda.forecast_period

<xarray.DataArray 'forecast_period' (time: 24)>
array([79664400000000000, 79668000000000000, 79671600000000000,
       79675200000000000, 79678800000000000, 79682400000000000,
       79686000000000000, 79689600000000000, 79693200000000000,
       79696800000000000, 79700400000000000, 79704000000000000,
       79707600000000000, 79711200000000000, 79714800000000000,
       79718400000000000, 79722000000000000, 79725600000000000,
       79729200000000000, 79732800000000000, 79736400000000000,
       79740000000000000, 79743600000000000, 79747200000000000],
      dtype='timedelta64[ns]')
Coordinates:
  * time                     (time) object 2006-07-23 01:00:00 ... 2006-07-24...
    forecast_period          (time) timedelta64[ns] 922 days 01:00:00 ... 923...
    forecast_reference_time  object 2004-01-01 00:00:00
Attributes:
    standard_name:  forecast_period

In [ ]:
ceda.time

<xarray.DataArray 'time' (time: 24)>
array([cftime.Datetime360Day(2006, 7, 23, 1, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 2, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 3, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 4, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 5, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 6, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 7, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 8, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 9, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 10, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 11, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 12, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 13, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 14, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 15, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 16, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 17, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 18, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 19, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 20, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 21, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 22, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 23, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 24, 0, 0, 0, 0, has_year_zero=True)],
      dtype=object)
Coordinates:
  * time                     (time) object 2006-07-23 01:00:00 ... 2006-07-24...
    forecast_period          (time) timedelta64[ns] 922 days 01:00:00 ... 923...
    forecast_reference_time  object 2004-01-01 00:00:00
Attributes:
    axis:           T
    standard_name:  time

In [ ]:
ceda.time.attrs

{'axis': 'T', 'standard_name': 'time'}

In [ ]:
ceda.coords['time']

<xarray.DataArray 'time' (time: 24)>
array([cftime.Datetime360Day(2006, 7, 23, 1, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 2, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 3, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 4, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 5, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 6, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 7, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 8, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 9, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 10, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 11, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 12, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 13, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 14, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 15, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 16, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 17, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 18, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 19, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 20, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 21, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 22, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 23, 23, 0, 0, 0, has_year_zero=True),
       cftime.Datetime360Day(2006, 7, 24, 0, 0, 0, 0, has_year_zero=True)],
      dtype=object)
Coordinates:
  * time                     (time) object 2006-07-23 01:00:00 ... 2006-07-24...
    forecast_period          (time) timedelta64[ns] 922 days 01:00:00 ... 923...
    forecast_reference_time  object 2004-01-01 00:00:00
Attributes:
    axis:           T
    standard_name:  time

By default, `xarray` decodes time data ([see this answer on StackOverflow](https://stackoverflow.com/a/45487851)), converting it to whichever `datatime` format it deems appropriate. However, this means losing any associated attribute data, which is not appropriate when needing to faithfully represent metadata across files and conversions.

In [ ]:
ceda.close()

In [ ]:
ceda = xr.open_dataset(
    os.path.join('../test_data/', filename),
    decode_times=False
)
ceda.time

<xarray.DataArray 'time' (time: 24)>
array([315889., 315890., 315891., 315892., 315893., 315894., 315895., 315896.,
       315897., 315898., 315899., 315900., 315901., 315902., 315903., 315904.,
       315905., 315906., 315907., 315908., 315909., 315910., 315911., 315912.])
Coordinates:
  * time                     (time) float64 3.159e+05 3.159e+05 ... 3.159e+05
    forecast_period          (time) float64 ...
    forecast_reference_time  float64 ...
Attributes:
    axis:           T
    units:          hours since 1970-01-01 00:00:00
    standard_name:  time
    calendar:       360_day

In [ ]:
ceda.close()


In [ ]:
filename = 'd20200128_diagnostic_0d_172800.nc'
filepath = os.path.join('../test_data/', filename)
nc0d_1 = xr.open_dataset(filepath)
nc0d_1

<xarray.Dataset>
Dimensions:               (time_series_300_1800: 2, zn: 121, z: 121, number_options: 664, kvp: 2)
Coordinates:
  * time_series_300_1800  (time_series_300_1800) float64 1.71e+05 1.728e+05
  * zn                    (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
  * z                     (z) float64 0.0 50.0 100.0 ... 5.9e+03 5.95e+03 6e+03
Dimensions without coordinates: number_options, kvp
Data variables: (12/23)
    VWP_mean              (time_series_300_1800) float32 ...
    LWP_mean              (time_series_300_1800) float32 ...
    RWP_mean              (time_series_300_1800) float32 ...
    VWP_max               (time_series_300_1800) float32 ...
    LWP_max               (time_series_300_1800) float32 ...
    RWP_max               (time_series_300_1800) float32 ...
    ...                    ...
    subke_max             (time_series_300_1800) float32 ...
    reske_min             (time_series_300_1800) float32 ...
    subke_min             (time_series_300_1800) float32 ...
    surface_precip_mean   (time_series_300_1800) float32 ...
    surface_precip_max    (time_series_300_1800) float32 ...
    options_database      (number_options, kvp) |S150 ...
Attributes:
    title:                         0d
    created:                       18/2/2022 7:59:55
    MONC time:                     172800.0
    MONC timestep:                 170650
    Diagnostic write frequency:    3600.0
    Previous diagnostic write at:  169200.0

In [ ]:
filename = 'd20200128_diagnostic_0d_176400.nc'
filepath = os.path.join('../test_data/', filename)
nc0d_2 = xr.open_dataset(filepath)
nc0d_2


<xarray.Dataset>
Dimensions:               (time_series_300_1800: 2, zn: 121, z: 121, number_options: 664, kvp: 2)
Coordinates:
  * time_series_300_1800  (time_series_300_1800) float64 1.746e+05 1.764e+05
  * zn                    (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
  * z                     (z) float64 0.0 50.0 100.0 ... 5.9e+03 5.95e+03 6e+03
Dimensions without coordinates: number_options, kvp
Data variables: (12/23)
    VWP_mean              (time_series_300_1800) float32 ...
    LWP_mean              (time_series_300_1800) float32 ...
    RWP_mean              (time_series_300_1800) float32 ...
    VWP_max               (time_series_300_1800) float32 ...
    LWP_max               (time_series_300_1800) float32 ...
    RWP_max               (time_series_300_1800) float32 ...
    ...                    ...
    subke_max             (time_series_300_1800) float32 ...
    reske_min             (time_series_300_1800) float32 ...
    subke_min             (time_series_300_1800) float32 ...
    surface_precip_mean   (time_series_300_1800) float32 ...
    surface_precip_max    (time_series_300_1800) float32 ...
    options_database      (number_options, kvp) |S150 ...
Attributes:
    title:                         0d
    created:                       18/2/2022 8:9:36
    MONC time:                     176400.0
    MONC timestep:                 173041
    Diagnostic write frequency:    3600.0
    Previous diagnostic write at:  172800.0

In [ ]:
both0d = xr.concat([nc0d_1, nc0d_2], dim='time_series_300_1800')
both0d['time_series_300_1800']

<xarray.DataArray 'time_series_300_1800' (time_series_300_1800: 4)>
array([171000., 172800., 174600., 176400.])
Coordinates:
  * time_series_300_1800  (time_series_300_1800) float64 1.71e+05 ... 1.764e+05

In [ ]:
both0d.attrs

{'title': '0d',
 'created': '18/2/2022 7:59:55',
 'MONC time': '172800.0',
 'MONC timestep': '170650',
 'Diagnostic write frequency': '3600.0',
 'Previous diagnostic write at': '169200.0'}

In [8]:
datetime.now().isoformat(timespec='seconds')

'2023-08-14T22:09:34'

In [6]:
datetime.now().strftime('%d/%m/%Y %H:%M:%S')

'14/08/2023 22:06:59'

In [ ]:
both0d.attrs['created'] = datetime.datetime.now().strftime('%d/%m/%Y %H:%M:%S')
both0d.attrs

{'title': '0d',
 'created': '27/07/2023 10:59:55',
 'MONC time': '172800.0',
 'MONC timestep': '170650',
 'Diagnostic write frequency': '3600.0',
 'Previous diagnostic write at': '169200.0'}

In [ ]:
[both0d.attrs.pop(k) for k in ['MONC time', 'MONC timestep',
                               'Previous diagnostic write at']]


['172800.0', '170650', '169200.0']

In [ ]:
both0d.attrs

{'title': '0d',
 'created': '27/07/2023 10:59:55',
 'Diagnostic write frequency': '3600.0'}

In [ ]:
options_db = {k: v
              for [k, v] in both0d.sel(time_series_300_1800=176400.).options_database.data}
options_db

{b'rad_interval': b'60',
 b'subsidence_local_q': b'true',
 b'passive_q': b'false',
 b'checkpoint_file': b'checkpoint_files/dephy_20200128_dump.nc',
 b'galilean_transformation': b'false',
 b'l_pgshd': b'true',
 b'smagorinsky_enabled': b'true',
 b'damping_enabled': b'true',
 b'gammam': b'19.30',
 b'gammah': b'12.0',
 b'symm_prob': b'false',
 b'pressure-terms_group_type': b'column',
 b'finalisation_stage_orderinga_34': b'conditional_diagnostics_column',
 b'finalisation_stage_orderinga_33': b'conditional_diagnostics_whole',
 b'finalisation_stage_orderinga_32': b'pstep',
 b'finalisation_stage_orderinga_31': b'tracers',
 b'finalisation_stage_orderinga_30': b'simplecloud',
 b'l_thref': b'false',
 b'finalisation_stage_orderinga_29': b'damping',
 b'finalisation_stage_orderinga_28': b'th_advection',
 b'finalisation_stage_orderinga_27': b'pw_advection',
 b'finalisation_stage_orderinga_26': b'profile_diagnostics',
 b'finalisation_stage_orderinga_25': b'viscosity',
 b'finalisation_stage_orderinga_2

In [ ]:
odb = nc0d_1.options_database
odb

AttributeError: 'Dataset' object has no attribute 'options_database'

In [ ]:
od_dims = nc0d_1.options_database.dims

In [ ]:
both0d.drop(labels='options_database')


<xarray.Dataset>
Dimensions:               (time_series_300_1800: 4, zn: 121, z: 121)
Coordinates:
  * time_series_300_1800  (time_series_300_1800) float64 1.71e+05 ... 1.764e+05
  * zn                    (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
  * z                     (z) float64 0.0 50.0 100.0 ... 5.9e+03 5.95e+03 6e+03
Data variables: (12/22)
    VWP_mean              (time_series_300_1800) float32 22.86 23.27 23.72 24.43
    LWP_mean              (time_series_300_1800) float32 4.418e-05 ... 0.0001801
    RWP_mean              (time_series_300_1800) float32 7.981e-11 ... 4.062e-10
    VWP_max               (time_series_300_1800) float32 24.93 25.31 25.98 26.82
    LWP_max               (time_series_300_1800) float32 0.1086 0.115 ... 0.1676
    RWP_max               (time_series_300_1800) float32 5.069e-06 ... 1.282e-05
    ...                    ...
    reske_max             (time_series_300_1800) float32 0.1981 ... 0.2672
    subke_max             (time_series_300_1800) float32 0.01103 ... 0.01763
    reske_min             (time_series_300_1800) float32 0.003478 ... 0.00364
    subke_min             (time_series_300_1800) float32 2.849e-05 ... 3.36e-05
    surface_precip_mean   (time_series_300_1800) float32 0.0 0.0 0.0 0.0
    surface_precip_max    (time_series_300_1800) float32 0.0 0.0 0.0 0.0
Attributes:
    title:                         0d
    created:                       18/2/2022 7:59:55
    MONC time:                     172800.0
    MONC timestep:                 170650
    Diagnostic write frequency:    3600.0
    Previous diagnostic write at:  169200.0

In [ ]:
both0d.assign(options_database=odb)

TypeError: variable 'options_database' has invalid type <class 'dict'>

In [ ]:
[ncfile.close() for ncfile in [nc0d_1, nc0d_2, both0d]]

[None, None, None]

In [ ]:
filename = 'd20200128_diagnostic_1d_43200.nc'
filepath = os.path.join('../test_data/', filename)
nc1d_1 = xr.open_dataset(filepath)
nc1d_1

<xarray.Dataset>
Dimensions:                (time_series_300_1800: 2, zn: 121, z: 121, number_options: 664, kvp: 2)
Coordinates:
  * time_series_300_1800   (time_series_300_1800) float64 4.14e+04 4.32e+04
  * zn                     (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
  * z                      (z) float64 0.0 50.0 100.0 ... 5.9e+03 5.95e+03 6e+03
Dimensions without coordinates: number_options, kvp
Data variables: (12/41)
    prefn                  (time_series_300_1800, zn) float32 ...
    rho                    (time_series_300_1800, z) float32 ...
    rhon                   (time_series_300_1800, zn) float32 ...
    thref                  (time_series_300_1800, zn) float32 ...
    thinit                 (time_series_300_1800, zn) float32 ...
    u_wind_mean            (time_series_300_1800, zn) float32 ...
    ...                     ...
    praut_mean             (time_series_300_1800, zn) float32 ...
    pracw_mean             (time_series_300_1800, zn) float32 ...
    prevp_mean             (time_series_300_1800, zn) float32 ...
    psedr_mean             (time_series_300_1800, zn) float32 ...
    psedl_mean             (time_series_300_1800, zn) float32 ...
    options_database       (number_options, kvp) |S150 ...
Attributes:
    title:                         1d
    created:                       18/2/2022 1:11:23
    MONC time:                     43200.0
    MONC timestep:                 49119
    Diagnostic write frequency:    3600.0
    Previous diagnostic write at:  39600.0

In [ ]:
filename = 'd20200128_diagnostic_1d_46800.nc'
filepath = os.path.join('../test_data/', filename)
nc1d_2 = xr.open_dataset(filepath)
nc1d_2

<xarray.Dataset>
Dimensions:                (time_series_300_1800: 2, zn: 121, z: 121, number_options: 664, kvp: 2)
Coordinates:
  * time_series_300_1800   (time_series_300_1800) float64 4.5e+04 4.68e+04
  * zn                     (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
  * z                      (z) float64 0.0 50.0 100.0 ... 5.9e+03 5.95e+03 6e+03
Dimensions without coordinates: number_options, kvp
Data variables: (12/41)
    prefn                  (time_series_300_1800, zn) float32 ...
    rho                    (time_series_300_1800, z) float32 ...
    rhon                   (time_series_300_1800, zn) float32 ...
    thref                  (time_series_300_1800, zn) float32 ...
    thinit                 (time_series_300_1800, zn) float32 ...
    u_wind_mean            (time_series_300_1800, zn) float32 ...
    ...                     ...
    praut_mean             (time_series_300_1800, zn) float32 ...
    pracw_mean             (time_series_300_1800, zn) float32 ...
    prevp_mean             (time_series_300_1800, zn) float32 ...
    psedr_mean             (time_series_300_1800, zn) float32 ...
    psedl_mean             (time_series_300_1800, zn) float32 ...
    options_database       (number_options, kvp) |S150 ...
Attributes:
    title:                         1d
    created:                       18/2/2022 1:23:14
    MONC time:                     46800.0
    MONC timestep:                 53065
    Diagnostic write frequency:    3600.0
    Previous diagnostic write at:  43200.0

In [ ]:
xr.combine_by_coords([nc1d_1, nc1d_2])

MergeError: combine_attrs='no_conflicts', but some values are not the same. Merging {'title': '1d', 'created': '18/2/2022 1:11:23', 'MONC time': '43200.0', 'MONC timestep': '49119', 'Diagnostic write frequency': '3600.0', 'Previous diagnostic write at': '39600.0'} with {'title': '1d', 'created': '18/2/2022 1:23:14', 'MONC time': '46800.0', 'MONC timestep': '53065', 'Diagnostic write frequency': '3600.0', 'Previous diagnostic write at': '43200.0'}

In [ ]:
t = datetime.datetime.now().strftime('%d/%m/%Y %H:%M:%S')  ## each file needs same values for each global attribute, to avoid conflicts.
for attributes in [nc1d_1.attrs, nc1d_2.attrs]:
    attributes['created'] = t
    [attributes.pop(k) for k in ['MONC time', 'MONC timestep',
                                   'Previous diagnostic write at']]


AttributeError: type object 'datetime.datetime' has no attribute 'datetime'

In [ ]:
xr.combine_by_coords([nc1d_1, nc1d_2])

MergeError: combine_attrs='no_conflicts', but some values are not the same. Merging {'title': '1d', 'created': '18/2/2022 1:11:23', 'MONC time': '43200.0', 'MONC timestep': '49119', 'Diagnostic write frequency': '3600.0', 'Previous diagnostic write at': '39600.0'} with {'title': '1d', 'created': '18/2/2022 1:23:14', 'MONC time': '46800.0', 'MONC timestep': '53065', 'Diagnostic write frequency': '3600.0', 'Previous diagnostic write at': '43200.0'}

In [ ]:
[f.close() for f in [nc1d_1, nc1d_2]]

[None, None]

In [ ]:
filename = 'd20200128_diagnostic_3d_313200.nc'
filepath = os.path.join('../test_data/', filename)
nc3d = xr.open_dataset(filepath)
nc3d

<xarray.Dataset>
Dimensions:                (time_series_1800_1800: 2, x: 1280, y: 1280, z: 121, zn: 121, number_options: 664, kvp: 2)
Coordinates:
  * time_series_1800_1800  (time_series_1800_1800) float64 3.114e+05 3.132e+05
  * zn                     (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
  * z                      (z) float64 0.0 50.0 100.0 ... 5.9e+03 5.95e+03 6e+03
Dimensions without coordinates: x, y, number_options, kvp
Data variables:
    w                      (time_series_1800_1800, x, y, z) float32 ...
    v                      (time_series_1800_1800, x, y, zn) float32 ...
    u                      (time_series_1800_1800, x, y, zn) float32 ...
    th                     (time_series_1800_1800, x, y, zn) float32 ...
    p                      (time_series_1800_1800, x, y, zn) float32 ...
    q_vapour               (time_series_1800_1800, x, y, zn) float32 ...
    q_cloud_liquid_mass    (time_series_1800_1800, x, y, zn) float32 ...
    q_rain_mass            (time_series_1800_1800, x, y, zn) float32 ...
    q_cloud_liquid_number  (time_series_1800_1800, x, y, zn) float32 ...
    q_rain_number          (time_series_1800_1800, x, y, zn) float32 ...
    options_database       (number_options, kvp) |S150 ...
Attributes:
    title:                         3d
    created:                       18/2/2022 16:16:1
    MONC time:                     313200.0
    MONC timestep:                 304297
    Diagnostic write frequency:    3600.0
    Previous diagnostic write at:  309600.0

In [ ]:
nc3d.sel(time_series_1800_1800=311400)


<xarray.Dataset>
Dimensions:                (x: 1280, y: 1280, z: 121, zn: 121, number_options: 664, kvp: 2)
Coordinates:
    time_series_1800_1800  float64 3.114e+05
  * zn                     (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
  * z                      (z) float64 0.0 50.0 100.0 ... 5.9e+03 5.95e+03 6e+03
Dimensions without coordinates: x, y, number_options, kvp
Data variables:
    w                      (x, y, z) float32 ...
    v                      (x, y, zn) float32 ...
    u                      (x, y, zn) float32 ...
    th                     (x, y, zn) float32 ...
    p                      (x, y, zn) float32 ...
    q_vapour               (x, y, zn) float32 ...
    q_cloud_liquid_mass    (x, y, zn) float32 ...
    q_rain_mass            (x, y, zn) float32 ...
    q_cloud_liquid_number  (x, y, zn) float32 ...
    q_rain_number          (x, y, zn) float32 ...
    options_database       (number_options, kvp) |S150 b'rad_interval' ... b'...
Attributes:
    title:                         3d
    created:                       18/2/2022 16:16:1
    MONC time:                     313200.0
    MONC timestep:                 304297
    Diagnostic write frequency:    3600.0
    Previous diagnostic write at:  309600.0

In [ ]:
nc3d.time_series_1800_1800.data

array([311400., 313200.])

In [ ]:
ds_by_t = [nc3d.sel(time_series_1800_1800=t)
           for t in nc3d.time_series_1800_1800.data]
len(ds_by_t)

2

In [ ]:
ds_by_t[0]

<xarray.Dataset>
Dimensions:                (x: 1280, y: 1280, z: 121, zn: 121, number_options: 664, kvp: 2)
Coordinates:
    time_series_1800_1800  float64 3.114e+05
  * zn                     (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
  * z                      (z) float64 0.0 50.0 100.0 ... 5.9e+03 5.95e+03 6e+03
Dimensions without coordinates: x, y, number_options, kvp
Data variables:
    w                      (x, y, z) float32 ...
    v                      (x, y, zn) float32 ...
    u                      (x, y, zn) float32 ...
    th                     (x, y, zn) float32 ...
    p                      (x, y, zn) float32 ...
    q_vapour               (x, y, zn) float32 ...
    q_cloud_liquid_mass    (x, y, zn) float32 ...
    q_rain_mass            (x, y, zn) float32 ...
    q_cloud_liquid_number  (x, y, zn) float32 ...
    q_rain_number          (x, y, zn) float32 ...
    options_database       (number_options, kvp) |S150 b'rad_interval' ... b'...
Attributes:
    title:                         3d
    created:                       18/2/2022 16:16:1
    MONC time:                     313200.0
    MONC timestep:                 304297
    Diagnostic write frequency:    3600.0
    Previous diagnostic write at:  309600.0

In [ ]:
ds_by_t[-1]


<xarray.Dataset>
Dimensions:                (x: 1280, y: 1280, z: 121, zn: 121, number_options: 664, kvp: 2)
Coordinates:
    time_series_1800_1800  float64 3.132e+05
  * zn                     (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
  * z                      (z) float64 0.0 50.0 100.0 ... 5.9e+03 5.95e+03 6e+03
Dimensions without coordinates: x, y, number_options, kvp
Data variables:
    w                      (x, y, z) float32 ...
    v                      (x, y, zn) float32 ...
    u                      (x, y, zn) float32 ...
    th                     (x, y, zn) float32 ...
    p                      (x, y, zn) float32 ...
    q_vapour               (x, y, zn) float32 ...
    q_cloud_liquid_mass    (x, y, zn) float32 ...
    q_rain_mass            (x, y, zn) float32 ...
    q_cloud_liquid_number  (x, y, zn) float32 ...
    q_rain_number          (x, y, zn) float32 ...
    options_database       (number_options, kvp) |S150 b'rad_interval' ... b'...
Attributes:
    title:                         3d
    created:                       18/2/2022 16:16:1
    MONC time:                     313200.0
    MONC timestep:                 304297
    Diagnostic write frequency:    3600.0
    Previous diagnostic write at:  309600.0

In [ ]:
[(obj, type(obj)) for obj in dir() if obj[0] != '_']

[('In', str),
 ('Out', str),
 ('app_dir', str),
 ('data_dir', str),
 ('date', str),
 ('date_match', str),
 ('datetime', str),
 ('dim_match', str),
 ('ds_by_t', str),
 ('end_time', str),
 ('exit', str),
 ('ext', str),
 ('filename', str),
 ('filepath', str),
 ('get_ipython', str),
 ('glob', str),
 ('k', str),
 ('monc_date', str),
 ('monc_day', str),
 ('monc_month', str),
 ('monc_year', str),
 ('n_dims', str),
 ('name_parts', str),
 ('nc1d_1', str),
 ('nc3d', str),
 ('os', str),
 ('part', str),
 ('quit', str),
 ('re', str),
 ('t_match', str),
 ('target_dir', str),
 ('timedelta', str),
 ('xr', str)]

In [ ]:
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'import xarray as xr\nimport os\nfrom glob import glob\n# import datetime\nimport re\nfrom datetime import datetime, date, timedelta',
  'app_dir = os.getcwd()\napp_dir',
  "data_dir = os.path.join(os.path.dirname(app_dir), 'test_data')\ndata_dir",
  'os.path.exists(data_dir)',
  "target_dir =  os.path.join(os.path.dirname(data_dir), f'{os.path.basename(data_dir)}+processed')\ntarget_dir",
  "filename = 'd20200128_diagnostic_1d_43200.nc'\n\nname_parts = re.split('[_\\.]', filename)\nprint(name_parts)\next = name_parts.pop()\nif ext != 'nc':\n    raise ValueError('Not a NetCDF file.')\nfor part in name_parts:\n    print(part)\n    if part == 'diagnostic':\n        continue\n    date_match = re.fullmatch('d[0-9]{8

In [ ]:
import gc
for obj in gc.get_objects():
    if isinstance(obj, xr.Dataset):
        obj.close()


In [205]:
nc0d_1.close()
nc1d_1.close()

In [ ]:
filenames = [
    'd20200128_diagnostic_0d_172800.nc', 
    'd20200128_diagnostic_1d_43200.nc'
]
filepath = os.path.join('../test_data/', filename)
ncfs = [xr.open_dataset(os.path.join('../test_data/', f)) for f in filenames]


In [ ]:
len(ncfs)

2

In [ ]:
odb = ncfs[0].options_database

In [ ]:
len(odb)

664

In [ ]:
for i, ds in enumerate(ncfs):
    print('options_database' in ncfs[i].variables)
    ncfs[i] = ds.drop_vars('options_database')
    print('options_database' in ncfs[i].variables)


True
False
True
False


In [ ]:
['options_database' in ds.variables for ds in ncfs]

[False, False]

In [ ]:
ncfs[0]

<xarray.Dataset>
Dimensions:               (time_series_300_1800: 2, zn: 121, z: 121)
Coordinates:
  * time_series_300_1800  (time_series_300_1800) float64 1.71e+05 1.728e+05
  * zn                    (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
  * z                     (z) float64 0.0 50.0 100.0 ... 5.9e+03 5.95e+03 6e+03
Data variables: (12/22)
    VWP_mean              (time_series_300_1800) float32 ...
    LWP_mean              (time_series_300_1800) float32 ...
    RWP_mean              (time_series_300_1800) float32 ...
    VWP_max               (time_series_300_1800) float32 ...
    LWP_max               (time_series_300_1800) float32 ...
    RWP_max               (time_series_300_1800) float32 ...
    ...                    ...
    reske_max             (time_series_300_1800) float32 ...
    subke_max             (time_series_300_1800) float32 ...
    reske_min             (time_series_300_1800) float32 ...
    subke_min             (time_series_300_1800) float32 ...
    surface_precip_mean   (time_series_300_1800) float32 ...
    surface_precip_max    (time_series_300_1800) float32 ...
Attributes:
    title:                         0d
    created:                       18/2/2022 7:59:55
    MONC time:                     172800.0
    MONC timestep:                 170650
    Diagnostic write frequency:    3600.0
    Previous diagnostic write at:  169200.0

In [ ]:
ncfs[1]

<xarray.Dataset>
Dimensions:                (time_series_300_1800: 2, zn: 121, z: 121)
Coordinates:
  * time_series_300_1800   (time_series_300_1800) float64 4.14e+04 4.32e+04
  * zn                     (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
  * z                      (z) float64 0.0 50.0 100.0 ... 5.9e+03 5.95e+03 6e+03
Data variables: (12/40)
    prefn                  (time_series_300_1800, zn) float32 ...
    rho                    (time_series_300_1800, z) float32 ...
    rhon                   (time_series_300_1800, zn) float32 ...
    thref                  (time_series_300_1800, zn) float32 ...
    thinit                 (time_series_300_1800, zn) float32 ...
    u_wind_mean            (time_series_300_1800, zn) float32 ...
    ...                     ...
    pcond_mean             (time_series_300_1800, zn) float32 ...
    praut_mean             (time_series_300_1800, zn) float32 ...
    pracw_mean             (time_series_300_1800, zn) float32 ...
    prevp_mean             (time_series_300_1800, zn) float32 ...
    psedr_mean             (time_series_300_1800, zn) float32 ...
    psedl_mean             (time_series_300_1800, zn) float32 ...
Attributes:
    title:                         1d
    created:                       18/2/2022 1:11:23
    MONC time:                     43200.0
    MONC timestep:                 49119
    Diagnostic write frequency:    3600.0
    Previous diagnostic write at:  39600.0

In [ ]:
ncfs[0].attrs

{'title': '0d',
 'created': '18/2/2022 7:59:55',
 'MONC time': '172800.0',
 'MONC timestep': '170650',
 'Diagnostic write frequency': '3600.0',
 'Previous diagnostic write at': '169200.0'}

In [ ]:
t = datetime.datetime.now().strftime('%d/%m/%Y %H:%M:%S')
for i, ds in enumerate(ncfs):
    for a in ['title', 'MONC time', 'MONC timestep', 'Previous diagnostic write at']:
        if a in ds.attrs: ds.attrs.pop(a)
    ds.attrs['created'] = t

AttributeError: type object 'datetime.datetime' has no attribute 'datetime'

In [ ]:
ncfs[0]

<xarray.Dataset>
Dimensions:               (time_series_300_1800: 2, zn: 121, z: 121)
Coordinates:
  * time_series_300_1800  (time_series_300_1800) float64 1.71e+05 1.728e+05
  * zn                    (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
  * z                     (z) float64 0.0 50.0 100.0 ... 5.9e+03 5.95e+03 6e+03
Data variables: (12/22)
    VWP_mean              (time_series_300_1800) float32 22.86 23.27
    LWP_mean              (time_series_300_1800) float32 4.418e-05 8.038e-05
    RWP_mean              (time_series_300_1800) float32 7.981e-11 7.887e-11
    VWP_max               (time_series_300_1800) float32 24.93 25.31
    LWP_max               (time_series_300_1800) float32 0.1086 0.115
    RWP_max               (time_series_300_1800) float32 5.069e-06 4.296e-06
    ...                    ...
    reske_max             (time_series_300_1800) float32 0.1981 0.2294
    subke_max             (time_series_300_1800) float32 0.01103 0.01095
    reske_min             (time_series_300_1800) float32 0.003478 0.003548
    subke_min             (time_series_300_1800) float32 2.849e-05 2.887e-05
    surface_precip_mean   (time_series_300_1800) float32 0.0 0.0
    surface_precip_max    (time_series_300_1800) float32 0.0 0.0
Attributes:
    title:                         0d
    created:                       18/2/2022 7:59:55
    MONC time:                     172800.0
    MONC timestep:                 170650
    Diagnostic write frequency:    3600.0
    Previous diagnostic write at:  169200.0

In [ ]:
xr.concat(ncfs, dim='time_series_300_1800', )


ValueError: 'VWP_mean' is not present in all datasets.

In [ ]:
xr.combine_by_coords(ncfs)

MergeError: combine_attrs='no_conflicts', but some values are not the same. Merging {'title': '0d', 'created': '18/2/2022 7:59:55', 'MONC time': '172800.0', 'MONC timestep': '170650', 'Diagnostic write frequency': '3600.0', 'Previous diagnostic write at': '169200.0'} with {'title': '1d', 'created': '18/2/2022 1:11:23', 'MONC time': '43200.0', 'MONC timestep': '49119', 'Diagnostic write frequency': '3600.0', 'Previous diagnostic write at': '39600.0'}

In [ ]:
xr.merge(ncfs)

<xarray.Dataset>
Dimensions:                (time_series_300_1800: 4, zn: 121, z: 121)
Coordinates:
  * time_series_300_1800   (time_series_300_1800) float64 4.14e+04 ... 1.728e+05
  * zn                     (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
  * z                      (z) float64 0.0 50.0 100.0 ... 5.9e+03 5.95e+03 6e+03
Data variables: (12/62)
    VWP_mean               (time_series_300_1800) float32 nan nan 22.86 23.27
    LWP_mean               (time_series_300_1800) float32 nan nan ... 8.038e-05
    RWP_mean               (time_series_300_1800) float32 nan nan ... 7.887e-11
    VWP_max                (time_series_300_1800) float32 nan nan 24.93 25.31
    LWP_max                (time_series_300_1800) float32 nan nan 0.1086 0.115
    RWP_max                (time_series_300_1800) float32 nan nan ... 4.296e-06
    ...                     ...
    pcond_mean             (time_series_300_1800, zn) float32 0.0 0.0 ... nan
    praut_mean             (time_series_300_1800, zn) float32 0.0 0.0 ... nan
    pracw_mean             (time_series_300_1800, zn) float32 0.0 0.0 ... nan
    prevp_mean             (time_series_300_1800, zn) float32 0.0 ... nan
    psedr_mean             (time_series_300_1800, zn) float32 0.0 ... nan
    psedl_mean             (time_series_300_1800, zn) float32 0.0 0.0 ... nan
Attributes:
    title:                         0d
    created:                       18/2/2022 7:59:55
    MONC time:                     172800.0
    MONC timestep:                 170650
    Diagnostic write frequency:    3600.0
    Previous diagnostic write at:  169200.0

So `xarray.Dataset.merge` and `xarray.Dataset.combine_by_coords` both work, but of course require matching timepoints!

In [ ]:
ncfs[0].z

<xarray.DataArray 'z' (z: 121)>
array([   0.,   50.,  100.,  150.,  200.,  250.,  300.,  350.,  400.,  450.,
        500.,  550.,  600.,  650.,  700.,  750.,  800.,  850.,  900.,  950.,
       1000., 1050., 1100., 1150., 1200., 1250., 1300., 1350., 1400., 1450.,
       1500., 1550., 1600., 1650., 1700., 1750., 1800., 1850., 1900., 1950.,
       2000., 2050., 2100., 2150., 2200., 2250., 2300., 2350., 2400., 2450.,
       2500., 2550., 2600., 2650., 2700., 2750., 2800., 2850., 2900., 2950.,
       3000., 3050., 3100., 3150., 3200., 3250., 3300., 3350., 3400., 3450.,
       3500., 3550., 3600., 3650., 3700., 3750., 3800., 3850., 3900., 3950.,
       4000., 4050., 4100., 4150., 4200., 4250., 4300., 4350., 4400., 4450.,
       4500., 4550., 4600., 4650., 4700., 4750., 4800., 4850., 4900., 4950.,
       5000., 5050., 5100., 5150., 5200., 5250., 5300., 5350., 5400., 5450.,
       5500., 5550., 5600., 5650., 5700., 5750., 5800., 5850., 5900., 5950.,
       6000.])
Coordinates:
  * z        (z) float64 0.0 50.0 100.0 150.0 ... 5.9e+03 5.95e+03 6e+03

In [ ]:
ncfs[0].zn

<xarray.DataArray 'zn' (zn: 121)>
array([ -25.,   25.,   75.,  125.,  175.,  225.,  275.,  325.,  375.,  425.,
        475.,  525.,  575.,  625.,  675.,  725.,  775.,  825.,  875.,  925.,
        975., 1025., 1075., 1125., 1175., 1225., 1275., 1325., 1375., 1425.,
       1475., 1525., 1575., 1625., 1675., 1725., 1775., 1825., 1875., 1925.,
       1975., 2025., 2075., 2125., 2175., 2225., 2275., 2325., 2375., 2425.,
       2475., 2525., 2575., 2625., 2675., 2725., 2775., 2825., 2875., 2925.,
       2975., 3025., 3075., 3125., 3175., 3225., 3275., 3325., 3375., 3425.,
       3475., 3525., 3575., 3625., 3675., 3725., 3775., 3825., 3875., 3925.,
       3975., 4025., 4075., 4125., 4175., 4225., 4275., 4325., 4375., 4425.,
       4475., 4525., 4575., 4625., 4675., 4725., 4775., 4825., 4875., 4925.,
       4975., 5025., 5075., 5125., 5175., 5225., 5275., 5325., 5375., 5425.,
       5475., 5525., 5575., 5625., 5675., 5725., 5775., 5825., 5875., 5925.,
       5975.])
Coordinates:
  * zn       (zn) float64 -25.0 25.0 75.0 ... 5.875e+03 5.925e+03 5.975e+03

In [ ]:
for k, v in list({k.decode():v.decode() for [k,v] in odb.data}.items()):
    if ('z0' in k):
        print(k, v)

z0th 0.00020
z0 0.00020


In [ ]:
options_db['z0th']

0.0002

In [ ]:
cfunits.Units('m s**-1').formatted()

'm.s-1'

In [ ]:
ncfs[0].time_series_300_1800

<xarray.DataArray 'time_series_300_1800' (time_series_300_1800: 2)>
array([171000., 172800.])
Coordinates:
  * time_series_300_1800  (time_series_300_1800) float64 1.71e+05 1.728e+05

In [ ]:
datetime.fromordinal(315960)

datetime.datetime(866, 1, 25, 0, 0)

In [ ]:
datetime.fromtimestamp(315960)


datetime.datetime(1970, 1, 4, 16, 46)

In [ ]:
datetime.utcfromtimestamp(315960)

datetime.datetime(1970, 1, 4, 15, 46)

In [ ]:
315960/(24*3600)

3.6569444444444446

In [ ]:
315960/365.25

865.0513347022587

In [ ]:
timedelta(seconds=315960)


datetime.timedelta(days=3, seconds=56760)

In [ ]:
datetime(year=2020, month=1, day=25, hour=0, minute=0, second=0)

datetime.datetime(2020, 1, 25, 0, 0)

In [ ]:
datetime(year=2020, month=1, day=25, hour=0, minute=0,
                  second=0) + timedelta(seconds=315960)

datetime.datetime(2020, 1, 28, 15, 46)

In [ ]:
os.getcwd()

'/Users/deepthought/Library/CloudStorage/OneDrive-UniversityofLeeds/EUREC4A/Code/CFizer/dev'

In [ ]:
glob(pathname='*.nc', root_dir='../test_data/')

['d20200128_diagnostic_3d_313200.nc',
 'd20200128_diagnostic_0d_176400.nc',
 'd20200128_diagnostic_0d_172800.nc',
 'd20200128_diagnostic_2d_176400.nc',
 'eurec4a_20200128_first.nc',
 'd20200128_diagnostic_1d_46800.nc',
 'd20200128_diagnostic_2d_172800.nc',
 'd20200128_diagnostic_1d_43200.nc']

In [102]:
from difflib import SequenceMatcher

In [ ]:
os.chdir(data_dir)

In [ ]:
files =  glob('*1d*.nc')
sm = SequenceMatcher(a=files[0], b=files[1])

In [ ]:
match = sm.find_longest_match()

In [ ]:
match.a

0

In [ ]:
files[0][match.a:match.a+match.size]

'd20200128_diagnostic_1d_4'

In [ ]:
files[1][match.a:match.a+match.size]

'd20200128_diagnostic_1d_4'

In [ ]:
files =  glob('*diagnostic*.nc')
s = None
for f in files:
    if s is None:
        s = f
    else:
        match = SequenceMatcher(a=s, b=f).find_longest_match()
        s = s[match.a: match.a + match.size]
s

'd20200128_diagnostic_'

In [ ]:
os.path.abspath(data_dir)

'/Users/deepthought/Library/CloudStorage/OneDrive-UniversityofLeeds/EUREC4A/Code/CFizer/test_data'

In [ ]:
SequenceMatcher(a='spam', b='').find_longest_match()

Match(a=0, b=0, size=0)

In [ ]:
filepath = os.path.join(data_dir, 'd20200128_diagnostic_3d_313200.nc')
nc3d = xr.open_dataset(filepath)
print(nc3d.dims)
print(nc3d.coords)
print([k.data for [k, v] in nc3d.options_database])


Frozen({'time_series_1800_1800': 2, 'x': 1280, 'y': 1280, 'z': 121, 'zn': 121, 'number_options': 664, 'kvp': 2})
Coordinates:
  * time_series_1800_1800  (time_series_1800_1800) float64 3.114e+05 3.132e+05
  * zn                     (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
  * z                      (z) float64 0.0 50.0 100.0 ... 5.9e+03 5.95e+03 6e+03
[array(b'rad_interval', dtype='|S150'), array(b'subsidence_local_q', dtype='|S150'), array(b'passive_q', dtype='|S150'), array(b'checkpoint_file', dtype='|S150'), array(b'galilean_transformation', dtype='|S150'), array(b'l_pgshd', dtype='|S150'), array(b'smagorinsky_enabled', dtype='|S150'), array(b'damping_enabled', dtype='|S150'), array(b'gammam', dtype='|S150'), array(b'gammah', dtype='|S150'), array(b'symm_prob', dtype='|S150'), array(b'pressure-terms_group_type', dtype='|S150'), array(b'finalisation_stage_orderinga_34', dtype='|S150'), array(b'finalisation_stage_orderinga_33', dtype='|S150'), array(b'finalisation_stage_or

In [ ]:
k, v = nc3d.options_database.data[0]
{k.decode(): v.decode()}

{'rad_interval': '60'}

In [ ]:
nc3d

<xarray.Dataset>
Dimensions:                (time_series_1800_1800: 2, x: 1280, y: 1280, z: 121, zn: 121, number_options: 664, kvp: 2)
Coordinates:
  * time_series_1800_1800  (time_series_1800_1800) float64 3.114e+05 3.132e+05
  * zn                     (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
  * z                      (z) float64 0.0 50.0 100.0 ... 5.9e+03 5.95e+03 6e+03
Dimensions without coordinates: x, y, number_options, kvp
Data variables:
    w                      (time_series_1800_1800, x, y, z) float32 ...
    v                      (time_series_1800_1800, x, y, zn) float32 ...
    u                      (time_series_1800_1800, x, y, zn) float32 ...
    th                     (time_series_1800_1800, x, y, zn) float32 ...
    p                      (time_series_1800_1800, x, y, zn) float32 ...
    q_vapour               (time_series_1800_1800, x, y, zn) float32 ...
    q_cloud_liquid_mass    (time_series_1800_1800, x, y, zn) float32 ...
    q_rain_mass            (time_series_1800_1800, x, y, zn) float32 ...
    q_cloud_liquid_number  (time_series_1800_1800, x, y, zn) float32 ...
    q_rain_number          (time_series_1800_1800, x, y, zn) float32 ...
    options_database       (number_options, kvp) |S150 b'rad_interval' ... b'...
Attributes:
    title:                         3d
    created:                       18/2/2022 16:16:1
    MONC time:                     313200.0
    MONC timestep:                 304297
    Diagnostic write frequency:    3600.0
    Previous diagnostic write at:  309600.0

In [ ]:
nc3d.x

<xarray.DataArray 'x' (x: 1280)>
array([   0,    1,    2, ..., 1277, 1278, 1279])
Dimensions without coordinates: x

In [ ]:
[k.decode() for [k,v] in nc3d.options_database.data if 'x' in k.decode()]

['cfl_cvismax',
 'cltop_longwave_flux',
 'fixed_cloud_re',
 'q_coordinates_x',
 'fixed_ice_re',
 'max_iterations',
 'l_subseds_maxv',
 'relax_to_initial_v_profile',
 'dxx',
 'surface_vapour_mixing_ratio',
 'max_change_buoyancy_flux',
 'clbase_longwave_flux',
 'l_sed_eulexp',
 'rate_change_geostrophic_wind_x',
 'x_size',
 'barrier_x',
 'cfl_rincmax',
 'l_solar_fixed',
 'fix_ugal',
 'max_mu',
 'solar_fixed',
 'sec_fixed',
 'l_fix_re',
 'rmlmax',
 'setfluxlook_enabled',
 'cfl_cvelmax',
 'cfl_dtmmax',
 'fix_vgal',
 'lwrad_exponential_enabled',
 'geostrophic_wind_rate_of_change_in_x',
 'use_surface_fluxes',
 'rotate_xy',
 'relax_to_initial_u_profile',
 'max_sed_length',
 'fix_mu',
 'max_height_cloud',
 'flux_budget_enabled',
 'max_step_length',
 'surface_geostrophic_wind_x',
 'fixed_cloud_number',
 'longwave_exp_decay',
 'relax_to_initial_theta_profile',
 'cfl_vt_max']

In [ ]:
dx = int([v.decode() for [k, v] in nc3d.options_database.data if k.decode() == 'dxx'][0])
dx

100

In [ ]:
x_range = [(x * dx + dx/2.) for x in range(nc3d.dims['x'])]
x_range

[50.0,
 150.0,
 250.0,
 350.0,
 450.0,
 550.0,
 650.0,
 750.0,
 850.0,
 950.0,
 1050.0,
 1150.0,
 1250.0,
 1350.0,
 1450.0,
 1550.0,
 1650.0,
 1750.0,
 1850.0,
 1950.0,
 2050.0,
 2150.0,
 2250.0,
 2350.0,
 2450.0,
 2550.0,
 2650.0,
 2750.0,
 2850.0,
 2950.0,
 3050.0,
 3150.0,
 3250.0,
 3350.0,
 3450.0,
 3550.0,
 3650.0,
 3750.0,
 3850.0,
 3950.0,
 4050.0,
 4150.0,
 4250.0,
 4350.0,
 4450.0,
 4550.0,
 4650.0,
 4750.0,
 4850.0,
 4950.0,
 5050.0,
 5150.0,
 5250.0,
 5350.0,
 5450.0,
 5550.0,
 5650.0,
 5750.0,
 5850.0,
 5950.0,
 6050.0,
 6150.0,
 6250.0,
 6350.0,
 6450.0,
 6550.0,
 6650.0,
 6750.0,
 6850.0,
 6950.0,
 7050.0,
 7150.0,
 7250.0,
 7350.0,
 7450.0,
 7550.0,
 7650.0,
 7750.0,
 7850.0,
 7950.0,
 8050.0,
 8150.0,
 8250.0,
 8350.0,
 8450.0,
 8550.0,
 8650.0,
 8750.0,
 8850.0,
 8950.0,
 9050.0,
 9150.0,
 9250.0,
 9350.0,
 9450.0,
 9550.0,
 9650.0,
 9750.0,
 9850.0,
 9950.0,
 10050.0,
 10150.0,
 10250.0,
 10350.0,
 10450.0,
 10550.0,
 10650.0,
 10750.0,
 10850.0,
 10950.0,
 11050.0,
 

In [ ]:
x_var = xr.Variable(dims='x', attrs={'units': 'm'}, data=x_range)
x_var

<xarray.Variable (x: 1280)>
array([5.0000e+01, 1.5000e+02, 2.5000e+02, ..., 1.2775e+05, 1.2785e+05,
       1.2795e+05])
Attributes:
    units:    m

In [ ]:
x_array = xr.DataArray(
    data=x_range,
    dims=["x"],
    coords={'x': x_var},
    attrs=dict(
        long_name="x position",
        units="m",
    ), 
    name='x'
    )
x_array

<xarray.DataArray 'x' (x: 1280)>
array([5.0000e+01, 1.5000e+02, 2.5000e+02, ..., 1.2775e+05, 1.2785e+05,
       1.2795e+05])
Coordinates:
  * x        (x) float64 50.0 150.0 250.0 350.0 ... 1.278e+05 1.278e+05 1.28e+05
Attributes:
    long_name:  x position
    units:      m

In [ ]:
nc3d.z

<xarray.DataArray 'z' (z: 121)>
array([   0.,   50.,  100.,  150.,  200.,  250.,  300.,  350.,  400.,  450.,
        500.,  550.,  600.,  650.,  700.,  750.,  800.,  850.,  900.,  950.,
       1000., 1050., 1100., 1150., 1200., 1250., 1300., 1350., 1400., 1450.,
       1500., 1550., 1600., 1650., 1700., 1750., 1800., 1850., 1900., 1950.,
       2000., 2050., 2100., 2150., 2200., 2250., 2300., 2350., 2400., 2450.,
       2500., 2550., 2600., 2650., 2700., 2750., 2800., 2850., 2900., 2950.,
       3000., 3050., 3100., 3150., 3200., 3250., 3300., 3350., 3400., 3450.,
       3500., 3550., 3600., 3650., 3700., 3750., 3800., 3850., 3900., 3950.,
       4000., 4050., 4100., 4150., 4200., 4250., 4300., 4350., 4400., 4450.,
       4500., 4550., 4600., 4650., 4700., 4750., 4800., 4850., 4900., 4950.,
       5000., 5050., 5100., 5150., 5200., 5250., 5300., 5350., 5400., 5450.,
       5500., 5550., 5600., 5650., 5700., 5750., 5800., 5850., 5900., 5950.,
       6000.])
Coordinates:
  * z        (z) float64 0.0 50.0 100.0 150.0 ... 5.9e+03 5.95e+03 6e+03

In [ ]:
nc3d.z.coords

Coordinates:
  * z        (z) float64 0.0 50.0 100.0 150.0 ... 5.9e+03 5.95e+03 6e+03

In [ ]:
nc3d.z.coords.dims

('z',)

In [ ]:
nc3d.z.coords.values

<bound method Mapping.values of Coordinates:
  * z        (z) float64 0.0 50.0 100.0 150.0 ... 5.9e+03 5.95e+03 6e+03>

In [ ]:
nc3d.z.data

array([   0.,   50.,  100.,  150.,  200.,  250.,  300.,  350.,  400.,
        450.,  500.,  550.,  600.,  650.,  700.,  750.,  800.,  850.,
        900.,  950., 1000., 1050., 1100., 1150., 1200., 1250., 1300.,
       1350., 1400., 1450., 1500., 1550., 1600., 1650., 1700., 1750.,
       1800., 1850., 1900., 1950., 2000., 2050., 2100., 2150., 2200.,
       2250., 2300., 2350., 2400., 2450., 2500., 2550., 2600., 2650.,
       2700., 2750., 2800., 2850., 2900., 2950., 3000., 3050., 3100.,
       3150., 3200., 3250., 3300., 3350., 3400., 3450., 3500., 3550.,
       3600., 3650., 3700., 3750., 3800., 3850., 3900., 3950., 4000.,
       4050., 4100., 4150., 4200., 4250., 4300., 4350., 4400., 4450.,
       4500., 4550., 4600., 4650., 4700., 4750., 4800., 4850., 4900.,
       4950., 5000., 5050., 5100., 5150., 5200., 5250., 5300., 5350.,
       5400., 5450., 5500., 5550., 5600., 5650., 5700., 5750., 5800.,
       5850., 5900., 5950., 6000.])

In [ ]:
nc3d.z.data.ndim

1

In [ ]:
type(nc3d.zn)

xarray.core.dataarray.DataArray

In [ ]:
nc3d = nc3d.assign(variables={'x': x_var})
nc3d

<xarray.Dataset>
Dimensions:                (time_series_1800_1800: 2, x: 1280, y: 1280, z: 121, zn: 121, number_options: 664, kvp: 2)
Coordinates:
  * time_series_1800_1800  (time_series_1800_1800) float64 3.114e+05 3.132e+05
  * zn                     (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
  * z                      (z) float64 0.0 50.0 100.0 ... 5.9e+03 5.95e+03 6e+03
  * x                      (x) float64 50.0 150.0 250.0 ... 1.278e+05 1.28e+05
Dimensions without coordinates: y, number_options, kvp
Data variables:
    w                      (time_series_1800_1800, x, y, z) float32 ...
    v                      (time_series_1800_1800, x, y, zn) float32 ...
    u                      (time_series_1800_1800, x, y, zn) float32 ...
    th                     (time_series_1800_1800, x, y, zn) float32 ...
    p                      (time_series_1800_1800, x, y, zn) float32 ...
    q_vapour               (time_series_1800_1800, x, y, zn) float32 ...
    q_cloud_liquid_mass    (time_series_1800_1800, x, y, zn) float32 ...
    q_rain_mass            (time_series_1800_1800, x, y, zn) float32 ...
    q_cloud_liquid_number  (time_series_1800_1800, x, y, zn) float32 ...
    q_rain_number          (time_series_1800_1800, x, y, zn) float32 ...
    options_database       (number_options, kvp) |S150 b'rad_interval' ... b'...
Attributes:
    title:                         3d
    created:                       18/2/2022 16:16:1
    MONC time:                     313200.0
    MONC timestep:                 304297
    Diagnostic write frequency:    3600.0
    Previous diagnostic write at:  309600.0

In [ ]:
nc3d.x

<xarray.DataArray 'x' (x: 1280)>
array([5.0000e+01, 1.5000e+02, 2.5000e+02, ..., 1.2775e+05, 1.2785e+05,
       1.2795e+05])
Coordinates:
  * x        (x) float64 50.0 150.0 250.0 350.0 ... 1.278e+05 1.278e+05 1.28e+05
Attributes:
    units:    m

In [ ]:
nc3d.z.assign_attrs({'units':'m'})

<xarray.DataArray 'z' (z: 121)>
array([   0.,   50.,  100.,  150.,  200.,  250.,  300.,  350.,  400.,  450.,
        500.,  550.,  600.,  650.,  700.,  750.,  800.,  850.,  900.,  950.,
       1000., 1050., 1100., 1150., 1200., 1250., 1300., 1350., 1400., 1450.,
       1500., 1550., 1600., 1650., 1700., 1750., 1800., 1850., 1900., 1950.,
       2000., 2050., 2100., 2150., 2200., 2250., 2300., 2350., 2400., 2450.,
       2500., 2550., 2600., 2650., 2700., 2750., 2800., 2850., 2900., 2950.,
       3000., 3050., 3100., 3150., 3200., 3250., 3300., 3350., 3400., 3450.,
       3500., 3550., 3600., 3650., 3700., 3750., 3800., 3850., 3900., 3950.,
       4000., 4050., 4100., 4150., 4200., 4250., 4300., 4350., 4400., 4450.,
       4500., 4550., 4600., 4650., 4700., 4750., 4800., 4850., 4900., 4950.,
       5000., 5050., 5100., 5150., 5200., 5250., 5300., 5350., 5400., 5450.,
       5500., 5550., 5600., 5650., 5700., 5750., 5800., 5850., 5900., 5950.,
       6000.])
Coordinates:
  * z        (z) float64 0.0 50.0 100.0 150.0 ... 5.9e+03 5.95e+03 6e+03
Attributes:
    units:    m

In [ ]:
nc3d.z

<xarray.DataArray 'z' (z: 121)>
array([   0.,   50.,  100.,  150.,  200.,  250.,  300.,  350.,  400.,  450.,
        500.,  550.,  600.,  650.,  700.,  750.,  800.,  850.,  900.,  950.,
       1000., 1050., 1100., 1150., 1200., 1250., 1300., 1350., 1400., 1450.,
       1500., 1550., 1600., 1650., 1700., 1750., 1800., 1850., 1900., 1950.,
       2000., 2050., 2100., 2150., 2200., 2250., 2300., 2350., 2400., 2450.,
       2500., 2550., 2600., 2650., 2700., 2750., 2800., 2850., 2900., 2950.,
       3000., 3050., 3100., 3150., 3200., 3250., 3300., 3350., 3400., 3450.,
       3500., 3550., 3600., 3650., 3700., 3750., 3800., 3850., 3900., 3950.,
       4000., 4050., 4100., 4150., 4200., 4250., 4300., 4350., 4400., 4450.,
       4500., 4550., 4600., 4650., 4700., 4750., 4800., 4850., 4900., 4950.,
       5000., 5050., 5100., 5150., 5200., 5250., 5300., 5350., 5400., 5450.,
       5500., 5550., 5600., 5650., 5700., 5750., 5800., 5850., 5900., 5950.,
       6000.])
Coordinates:
  * z        (z) float64 0.0 50.0 100.0 150.0 ... 5.9e+03 5.95e+03 6e+03

In [ ]:
nc3d.z.attrs['units'] = 'm'

In [ ]:
nc3d.z

<xarray.DataArray 'z' (z: 121)>
array([   0.,   50.,  100.,  150.,  200.,  250.,  300.,  350.,  400.,  450.,
        500.,  550.,  600.,  650.,  700.,  750.,  800.,  850.,  900.,  950.,
       1000., 1050., 1100., 1150., 1200., 1250., 1300., 1350., 1400., 1450.,
       1500., 1550., 1600., 1650., 1700., 1750., 1800., 1850., 1900., 1950.,
       2000., 2050., 2100., 2150., 2200., 2250., 2300., 2350., 2400., 2450.,
       2500., 2550., 2600., 2650., 2700., 2750., 2800., 2850., 2900., 2950.,
       3000., 3050., 3100., 3150., 3200., 3250., 3300., 3350., 3400., 3450.,
       3500., 3550., 3600., 3650., 3700., 3750., 3800., 3850., 3900., 3950.,
       4000., 4050., 4100., 4150., 4200., 4250., 4300., 4350., 4400., 4450.,
       4500., 4550., 4600., 4650., 4700., 4750., 4800., 4850., 4900., 4950.,
       5000., 5050., 5100., 5150., 5200., 5250., 5300., 5350., 5400., 5450.,
       5500., 5550., 5600., 5650., 5700., 5750., 5800., 5850., 5900., 5950.,
       6000.])
Coordinates:
  * z        (z) float64 0.0 50.0 100.0 150.0 ... 5.9e+03 5.95e+03 6e+03
Attributes:
    units:    m

In [ ]:
nc3d.coords['zn']

<xarray.DataArray 'zn' (zn: 121)>
array([ -25.,   25.,   75.,  125.,  175.,  225.,  275.,  325.,  375.,  425.,
        475.,  525.,  575.,  625.,  675.,  725.,  775.,  825.,  875.,  925.,
        975., 1025., 1075., 1125., 1175., 1225., 1275., 1325., 1375., 1425.,
       1475., 1525., 1575., 1625., 1675., 1725., 1775., 1825., 1875., 1925.,
       1975., 2025., 2075., 2125., 2175., 2225., 2275., 2325., 2375., 2425.,
       2475., 2525., 2575., 2625., 2675., 2725., 2775., 2825., 2875., 2925.,
       2975., 3025., 3075., 3125., 3175., 3225., 3275., 3325., 3375., 3425.,
       3475., 3525., 3575., 3625., 3675., 3725., 3775., 3825., 3875., 3925.,
       3975., 4025., 4075., 4125., 4175., 4225., 4275., 4325., 4375., 4425.,
       4475., 4525., 4575., 4625., 4675., 4725., 4775., 4825., 4875., 4925.,
       4975., 5025., 5075., 5125., 5175., 5225., 5275., 5325., 5375., 5425.,
       5475., 5525., 5575., 5625., 5675., 5725., 5775., 5825., 5875., 5925.,
       5975.])
Coordinates:
  * zn       (zn) float64 -25.0 25.0 75.0 ... 5.875e+03 5.925e+03 5.975e+03

In [ ]:
nc3d.coords['zn'].coords

Coordinates:
  * zn       (zn) float64 -25.0 25.0 75.0 ... 5.875e+03 5.925e+03 5.975e+03

In [ ]:
nc3d.coords['zn'].data

array([ -25.,   25.,   75.,  125.,  175.,  225.,  275.,  325.,  375.,
        425.,  475.,  525.,  575.,  625.,  675.,  725.,  775.,  825.,
        875.,  925.,  975., 1025., 1075., 1125., 1175., 1225., 1275.,
       1325., 1375., 1425., 1475., 1525., 1575., 1625., 1675., 1725.,
       1775., 1825., 1875., 1925., 1975., 2025., 2075., 2125., 2175.,
       2225., 2275., 2325., 2375., 2425., 2475., 2525., 2575., 2625.,
       2675., 2725., 2775., 2825., 2875., 2925., 2975., 3025., 3075.,
       3125., 3175., 3225., 3275., 3325., 3375., 3425., 3475., 3525.,
       3575., 3625., 3675., 3725., 3775., 3825., 3875., 3925., 3975.,
       4025., 4075., 4125., 4175., 4225., 4275., 4325., 4375., 4425.,
       4475., 4525., 4575., 4625., 4675., 4725., 4775., 4825., 4875.,
       4925., 4975., 5025., 5075., 5125., 5175., 5225., 5275., 5325.,
       5375., 5425., 5475., 5525., 5575., 5625., 5675., 5725., 5775.,
       5825., 5875., 5925., 5975.])

In [ ]:
nc3d.coords['zn'].coords == nc3d.coords['zn'].data

/var/folders/9y/725xtb1d3kg5d01vmypjtrmm0000gn/T/ipykernel_7304/2992028198.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  nc3d.coords['zn'].coords == nc3d.coords['zn'].data


False

In [ ]:
nc3d.coords

Coordinates:
  * time_series_1800_1800  (time_series_1800_1800) float64 3.114e+05 3.132e+05
  * zn                     (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
  * z                      (z) float64 0.0 50.0 100.0 ... 5.9e+03 5.95e+03 6e+03
  * x                      (x) float64 50.0 150.0 250.0 ... 1.278e+05 1.28e+05

In [ ]:
nc3d.dims

Frozen({'time_series_1800_1800': 2, 'x': 1280, 'y': 1280, 'z': 121, 'zn': 121, 'number_options': 664, 'kvp': 2})

In [ ]:
for dim, size in nc3d.dims.items():
    print(f'{dim} in coords: {dim in nc3d.coords}')

time_series_1800_1800 in coords: True
x in coords: True
y in coords: False
z in coords: True
zn in coords: True
number_options in coords: False
kvp in coords: False


In [ ]:
{k.decode():v.decode() for [k, v] in nc3d.options_database.data if k.decode() in ['dxx','dyy']}

{'dxx': '100', 'dyy': '100'}

In [ ]:
nc_ceda = xr.open_dataset(os.path.join(data_dir, 'CEDA_archived_output.nc'),decode_times=False)
nc_ceda

FileNotFoundError: [Errno 2] No such file or directory: b'/Users/deepthought/Library/CloudStorage/OneDrive-UniversityofLeeds/EUREC4A/Code/CFizer/test_data/CEDA_archived_output.nc'

In [ ]:
nc_ceda.coords

Coordinates:
  * time                     (time) float64 3.159e+05 3.159e+05 ... 3.159e+05
  * depth                    (depth) float32 0.05 0.225 0.675 2.0
  * latitude                 (latitude) float32 -45.5 -45.46 ... 39.46 39.5
  * longitude                (longitude) float32 335.5 335.6 ... 416.4 416.5
    forecast_period          (time) float64 ...
    forecast_reference_time  float64 ...

In [ ]:
nc_ceda.time

<xarray.DataArray 'time' (time: 24)>
array([315889., 315890., 315891., 315892., 315893., 315894., 315895., 315896.,
       315897., 315898., 315899., 315900., 315901., 315902., 315903., 315904.,
       315905., 315906., 315907., 315908., 315909., 315910., 315911., 315912.])
Coordinates:
  * time                     (time) float64 3.159e+05 3.159e+05 ... 3.159e+05
    forecast_period          (time) float64 ...
    forecast_reference_time  float64 ...
Attributes:
    axis:           T
    units:          hours since 1970-01-01 00:00:00
    standard_name:  time
    calendar:       360_day

In [ ]:
nc_ceda.time.coords

Coordinates:
  * time                     (time) float64 3.159e+05 3.159e+05 ... 3.159e+05
    forecast_period          (time) float64 ...
    forecast_reference_time  float64 ...

In [ ]:
nc_ceda.coords['time'].data

array([315889., 315890., 315891., 315892., 315893., 315894., 315895.,
       315896., 315897., 315898., 315899., 315900., 315901., 315902.,
       315903., 315904., 315905., 315906., 315907., 315908., 315909.,
       315910., 315911., 315912.])

In [ ]:
nc_ceda.coords['time'].indexes

Indexes:
    time     Index([315889.0, 315890.0, 315891.0, 315892.0, 315893.0, 315894.0, 315895.0,
       315896.0, 315897.0, 315898.0, 315899.0, 315900.0, 315901.0, 315902.0,
       315903.0, 315904.0, 315905.0, 315906.0, 315907.0, 315908.0, 315909.0,
       315910.0, 315911.0, 315912.0],
      dtype='float64', name='time')

In [ ]:
nc_ceda.coords['forecast_period'].coords

Coordinates:
  * time                     (time) float64 3.159e+05 3.159e+05 ... 3.159e+05
    forecast_period          (time) float64 ...
    forecast_reference_time  float64 ...

In [ ]:
nc_ceda.coords['forecast_period'].data

array([22129., 22130., 22131., 22132., 22133., 22134., 22135., 22136.,
       22137., 22138., 22139., 22140., 22141., 22142., 22143., 22144.,
       22145., 22146., 22147., 22148., 22149., 22150., 22151., 22152.])

In [ ]:
nc_ceda.coords['forecast_period'].indexes

Indexes:
    time     Index([315889.0, 315890.0, 315891.0, 315892.0, 315893.0, 315894.0, 315895.0,
       315896.0, 315897.0, 315898.0, 315899.0, 315900.0, 315901.0, 315902.0,
       315903.0, 315904.0, 315905.0, 315906.0, 315907.0, 315908.0, 315909.0,
       315910.0, 315911.0, 315912.0],
      dtype='float64', name='time')

In [ ]:
nc_ceda.close()

In [ ]:
nc3d.z

<xarray.DataArray 'z' (z: 121)>
array([   0.,   50.,  100.,  150.,  200.,  250.,  300.,  350.,  400.,  450.,
        500.,  550.,  600.,  650.,  700.,  750.,  800.,  850.,  900.,  950.,
       1000., 1050., 1100., 1150., 1200., 1250., 1300., 1350., 1400., 1450.,
       1500., 1550., 1600., 1650., 1700., 1750., 1800., 1850., 1900., 1950.,
       2000., 2050., 2100., 2150., 2200., 2250., 2300., 2350., 2400., 2450.,
       2500., 2550., 2600., 2650., 2700., 2750., 2800., 2850., 2900., 2950.,
       3000., 3050., 3100., 3150., 3200., 3250., 3300., 3350., 3400., 3450.,
       3500., 3550., 3600., 3650., 3700., 3750., 3800., 3850., 3900., 3950.,
       4000., 4050., 4100., 4150., 4200., 4250., 4300., 4350., 4400., 4450.,
       4500., 4550., 4600., 4650., 4700., 4750., 4800., 4850., 4900., 4950.,
       5000., 5050., 5100., 5150., 5200., 5250., 5300., 5350., 5400., 5450.,
       5500., 5550., 5600., 5650., 5700., 5750., 5800., 5850., 5900., 5950.,
       6000.])
Coordinates:
  * z        (z) float64 0.0 50.0 100.0 150.0 ... 5.9e+03 5.95e+03 6e+03

In [ ]:
nc3d.dims

Frozen({'time_series_1800_1800': 2, 'x': 1280, 'y': 1280, 'z': 121, 'zn': 121, 'number_options': 664, 'kvp': 2})

In [ ]:
odb_var = 'options_database'  # this could come from config file
odb_dims = ('number_options', 'kvp')
odb = {k.decode('utf-8'): v.decode('utf-8') 
                    for [k, v] in nc3d['options_database'].data}
odb['dxx']

'100'

In [ ]:
nc3d.x

<xarray.DataArray 'x' (x: 1280)>
array([   0,    1,    2, ..., 1277, 1278, 1279])
Dimensions without coordinates: x

In [ ]:
nc3d.zn

<xarray.DataArray 'zn' (zn: 121)>
array([ -25.,   25.,   75.,  125.,  175.,  225.,  275.,  325.,  375.,  425.,
        475.,  525.,  575.,  625.,  675.,  725.,  775.,  825.,  875.,  925.,
        975., 1025., 1075., 1125., 1175., 1225., 1275., 1325., 1375., 1425.,
       1475., 1525., 1575., 1625., 1675., 1725., 1775., 1825., 1875., 1925.,
       1975., 2025., 2075., 2125., 2175., 2225., 2275., 2325., 2375., 2425.,
       2475., 2525., 2575., 2625., 2675., 2725., 2775., 2825., 2875., 2925.,
       2975., 3025., 3075., 3125., 3175., 3225., 3275., 3325., 3375., 3425.,
       3475., 3525., 3575., 3625., 3675., 3725., 3775., 3825., 3875., 3925.,
       3975., 4025., 4075., 4125., 4175., 4225., 4275., 4325., 4375., 4425.,
       4475., 4525., 4575., 4625., 4675., 4725., 4775., 4825., 4875., 4925.,
       4975., 5025., 5075., 5125., 5175., 5225., 5275., 5325., 5375., 5425.,
       5475., 5525., 5575., 5625., 5675., 5725., 5775., 5825., 5875., 5925.,
       5975.])
Coordinates:
  * zn       (zn) float64 -25.0 25.0 75.0 ... 5.875e+03 5.925e+03 5.975e+03

In [ ]:
dephy = xr.open_dataset(os.path.join(data_dir, 'eurec4a_20200128_first.nc'), decode_times=False)
dephy

<xarray.Dataset>
Dimensions:                (time: 101, level: 250)
Coordinates:
  * time                   (time) float64 0.0 3.6e+03 ... 3.564e+05 3.6e+05
  * level                  (level) float64 0.0 20.0 40.33 ... 7.08e+04 7.2e+04
Data variables: (12/251)
    lat                    (time) float64 ...
    lon                    (time) float64 ...
    u_traj                 (time) float64 ...
    v_traj                 (time) float64 ...
    w_local                (time, level) float64 ...
    q_local                (time, level) float64 ...
    ...                     ...
    dtheta_ldy             (time, level) float64 ...
    u_g                    (time, level) float64 ...
    v_g                    (time, level) float64 ...
    origin_lon             float64 ...
    origin_lat             float64 ...
    origin_datetime        float64 ...
Attributes: (12/57)
    trajectory_type:                                        lagrangian
    velocity_method:                                        single_height_level
    velocity_method_kwargs_height:                          500.0
    domain_conventions:                                     CF-1.7
    domain_contact:                                         l.c.denby[at]leed...
    domain_era5_reference:                                  Hersbach, H., Bel...
    ...                                                     ...
    trajectory_version:                                     1.0.1
    sampling_method_gradient_method:                        regression
    sampling_method_advection_velocity_sampling_method:     domain_mean
    sampling_method_averaging_width:                        2.0
    sampling_method_time_sampling_method:                   domain_data
    sampling_method_mask:                                   ocean_only

In [ ]:
(dephy.origin_datetime.data, dephy.origin_datetime.units)

(array(315960.), 'seconds since 2020-01-25')

In [ ]:
dephy.origin_datetime.data.shape

()

In [ ]:
dephy.origin_datetime.data.size

1

In [ ]:
float(dephy.origin_datetime.data)

315960.0

In [ ]:
dephy.time.units

'seconds since 2020-01-25'

In [ ]:
dephy.time.units.split(' since ')

['seconds', '2020-01-25']

In [ ]:
find_ref_datetime = re.compile(pattern='(?<=since ).*')
find_ref_datetime.search(dephy.time.units)[0]

'2020-01-25'

In [ ]:
ref_datetime = datetime.fromisoformat(
    find_ref_datetime.search(dephy.time.units)[0]
)
ref_datetime

datetime.datetime(2020, 1, 25, 0, 0)

In [ ]:
ref_datetime_units = re.search(pattern='.*(?= since)', 
                               string=dephy.time.units)[0]
ref_datetime_units

'seconds'

In [ ]:
time_units = {
    'seconds'
}

In [ ]:
dephy.attrs['origin_datetime']

'2020-01-28T15:46:00'

In [ ]:
origin_datetime = datetime.fromisoformat(dephy.attrs['origin_datetime'])
origin_datetime

datetime.datetime(2020, 1, 28, 15, 46)

In [ ]:
timespan = timedelta(float(dephy.origin_datetime.data))
timespan

datetime.timedelta(days=315960)

In [ ]:
timedelta.__doc__

'Difference between two datetime values.\n\ntimedelta(days=0, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)\n\nAll arguments are optional and default to 0.\nArguments may be integers or floats, and may be positive or negative.'

In [ ]:
time_units = dict(
    days=0,
    seconds=0, 
    microseconds=0, 
    milliseconds=0, 
    minutes=0, 
    hours=0, 
    weeks=0
)

In [ ]:
time_units[ref_datetime_units] = float(dephy.origin_datetime.data)
time_units

{'days': 0,
 'seconds': 315960.0,
 'microseconds': 0,
 'milliseconds': 0,
 'minutes': 0,
 'hours': 0,
 'weeks': 0}

In [ ]:
timedelta(days=time_units['days'],
          seconds=time_units['seconds'],
          microseconds=time_units['microseconds'],
          milliseconds=time_units['milliseconds'],
          minutes=time_units['minutes'],
          hours=time_units['hours'],
          weeks=time_units['weeks'])

datetime.timedelta(days=3, seconds=56760)

In [ ]:
ref_datetime + timedelta(
    days=time_units['days'],
    seconds=time_units['seconds'],
    microseconds=time_units['microseconds'],
    milliseconds=time_units['milliseconds'],
    minutes=time_units['minutes'],
    hours=time_units['hours'],
    weeks=time_units['weeks']
) == origin_datetime

True

In [ ]:
float('blan')

ValueError: could not convert string to float: 'blan'

In [ ]:
astring = 43.9
astring.lower()

AttributeError: 'float' object has no attribute 'lower'

In [ ]:
datetime.fromisoformat('45')

ValueError: Invalid isoformat string: '45'

In [ ]:
'this does not have right split in it'.split('since')

['this does not have right split in it']

In [ ]:
first, second = 'this does not have right split in it'.split('since')

ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
dephy.variables['time']

<xarray.IndexVariable 'time' (time: 101)>
array([     0.,   3600.,   7200.,  10800.,  14400.,  18000.,  21600.,  25200.,
        28800.,  32400.,  36000.,  39600.,  43200.,  46800.,  50400.,  54000.,
        57600.,  61200.,  64800.,  68400.,  72000.,  75600.,  79200.,  82800.,
        86400.,  90000.,  93600.,  97200., 100800., 104400., 108000., 111600.,
       115200., 118800., 122400., 126000., 129600., 133200., 136800., 140400.,
       144000., 147600., 151200., 154800., 158400., 162000., 165600., 169200.,
       172800., 176400., 180000., 183600., 187200., 190800., 194400., 198000.,
       201600., 205200., 208800., 212400., 216000., 219600., 223200., 226800.,
       230400., 234000., 237600., 241200., 244800., 248400., 252000., 255600.,
       259200., 262800., 266400., 270000., 273600., 277200., 280800., 284400.,
       288000., 291600., 295200., 298800., 302400., 306000., 309600., 313200.,
       316800., 320400., 324000., 327600., 331200., 334800., 338400., 342000.,
       345600., 349200., 352800., 356400., 360000.])
Attributes:
    units:     seconds since 2020-01-25
    calendar:  proleptic_gregorian

In [ ]:
dephy.attrs['origin_datetime']

'2020-01-28T15:46:00'

In [5]:
t_unit = cfunits.Units(units='seconds since 2020-01-25', calendar='proleptic_gregorian')

In [6]:
t_unit.equals(cfunits.Units(units='s'))

False

In [7]:
t_unit.equals(cfunits.Units(units='s', calendar='proleptic_gregorian'))

False

In [8]:
t_unit.equals(cfunits.Units(units='seconds since 2020-01-25', calendar='standard'))

False

In [9]:
t_unit.equals(cfunits.Units(units='seconds since 2020-01-25', calendar='proleptic_gregorian'))

True

In [18]:
cfunits.Units(units='seconds since 2020-01-25', calendar='gregorian').equals(cfunits.Units(units='seconds since 2020-01-25', calendar='standard'))

True

In [20]:
cfunits.Units(units=t_unit.units).equivalent(cfunits.Units(units='s'))

False

In [24]:
print(t_unit.formatted().split(' since ')[0])
print('s'.split(' since ')[0])

s
s


In [14]:
cfunits.Units.conform([0, 3600, 7200], from_units=cfunits.Units(units='s since 2020-01-25', calendar='standard'), to_units=t_unit)

ValueError: Units are not convertible: <Units: s since 2020-01-25 standard>, <Units: seconds since 2020-01-25 proleptic_gregorian>

In [ ]:
t_unit.calendar

'proleptic_gregorian'

In [ ]:
t_unit.isvalid

True

In [ ]:
cfunits.Units('minutes since 2020-01-30').isvalid

True

In [ ]:
cfunits.Units('minutes since 2020-01-30').iscalendartime

False

In [ ]:
cfunits.Units('minutes since 2020-01-30').isreftime

True

In [4]:
cfunits.Units('minutes').isreftime

False

In [ ]:
cfunits.Units('minutes').istime

True

In [ ]:
cfunits.Units('minutes since 2020-01-30').formatted()

'min since 2020-01-30 00:00:00'

In [ ]:
cfunits.Units('minutes').formatted()

'min'

In [ ]:
cfunits.Units(calendar='proleptic_gregorian').isvalid

True

In [ ]:
cfunits.Units(calendar='proleptic_gregorian').isreftime

True

In [ ]:
t_unit.formatted()

's since 2020-01-25 00:00:00'

In [ ]:
t_unit.isvalid

True

In [ ]:
[a for a in dir(t_unit) if a[0] is not '_']

<>:1: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:1: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/var/folders/9y/725xtb1d3kg5d01vmypjtrmm0000gn/T/ipykernel_7304/1937667762.py:1: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  [a for a in dir(t_unit) if a[0] is not '_']


['calendar',
 'conform',
 'copy',
 'equals',
 'equivalent',
 'formatted',
 'has_offset',
 'iscalendartime',
 'isdimensionless',
 'islatitude',
 'islongitude',
 'ispressure',
 'isreftime',
 'istime',
 'isvalid',
 'log',
 'reason_notvalid',
 'reftime',
 'units']

In [ ]:
t_unit.reftime

cftime.datetime(2020, 1, 25, 0, 0, 0, 0, calendar='proleptic_gregorian', has_year_zero=True)

In [ ]:
t_unit.reftime.isoformat()

'2020-01-25T00:00:00'

In [ ]:
cal = cfunits.Units(calendar='standard')
time_unit = cfunits.Units(units='seconds since 2020-01-25')
t_unit = cfunits.Units(units=time_unit, calendar=cal)

In [ ]:
t_unit.formatted()

's since 2020-01-25 00:00:00'

In [ ]:
cal0 = None
cal1 = 'gregorian'
cal2 = 'standard'
unit_test = cfunits.Units(calendar=(cal0 or cal1 or cal2))

In [ ]:
unit_test.formatted()

In [ ]:
unit_test.calendar

'gregorian'

In [ ]:
cfunits.CALENDAR_STANDARD

AttributeError: module 'cfunits' has no attribute 'CALENDAR_STANDARD'

In [ ]:
toy = {'a': 'seconds'}
thing = toy['b'] or toy['a'] or 2

KeyError: 'b'

In [ ]:
{k.replace('rad_start_', ''):v for k, v in options_db.items() if 'rad_start' in k}

{'year': 2020.0, 'time': 0.0, 'day': 25.0}

In [ ]:
datetime(year=int(options_db['rad_start_year']), day=int(options_db['rad_start_day']), second=options_db['rad_start_time'])

TypeError: function missing required argument 'month' (pos 2)

In [ ]:
(datetime(year=2020, month=1, day=1) + timedelta(days=(32 - 1))).isoformat()


'2020-02-01T00:00:00'

In [ ]:
float(69.e3)

69000.0

In [ ]:
float('a')

ValueError: could not convert string to float: 'a'

In [ ]:
for ds in [nc1d_1, nc3d]:
    ds.close()

In [18]:
nc1d_1.close()

In [206]:
paths = [os.path.join(data_dir, f) for f in glob('*_??00.nc', root_dir=data_dir)]
dimensionality = {
    '3d': [],
    '2d': [],
    '0+1d': []
}
for path in paths:
    f = os.path.basename(path)
    if '_3d_' in f: 
        dimensionality['3d'].append(xr.open_dataset(path, decode_times=False))
        [print('3d:', f, dim) for dim in dimensionality['3d'][-1].dims if 'time' in dim]
    elif '_2d_' in f:
        dimensionality['2d'].append(xr.open_dataset(path, decode_times=False))
        [print('2d:', f, dim) for dim in dimensionality['2d'][-1].dims if 'time' in dim]
    else:
        dimensionality['0+1d'].append(xr.open_dataset(path, decode_times=False))
        [print('0+1d:', f, dim) for dim in dimensionality['0+1d'][-1].dims if 'time' in dim]
        
{k: v[0].data_vars for k, v in dimensionality.items()}

2d: d20200128_diagnostic_2d_3600.nc time_series_1800_1800
3d: d20200128_diagnostic_3d_3600.nc time_series_1800_1800
0+1d: d20200128_diagnostic_1d_7200.nc time_series_300_1800
0+1d: d20200128_diagnostic_0d_7200.nc time_series_300_1800
2d: d20200128_diagnostic_2d_7200.nc time_series_1800_1800
3d: d20200128_diagnostic_3d_7200.nc time_series_1800_1800
0+1d: d20200128_diagnostic_1d_3600.nc time_series_300_1800
0+1d: d20200128_diagnostic_0d_3600.nc time_series_300_1800


{'3d': Data variables:
     w                      (time_series_1800_1800, x, y, z) float32 ...
     v                      (time_series_1800_1800, x, y, zn) float32 ...
     u                      (time_series_1800_1800, x, y, zn) float32 ...
     th                     (time_series_1800_1800, x, y, zn) float32 ...
     p                      (time_series_1800_1800, x, y, zn) float32 ...
     q_vapour               (time_series_1800_1800, x, y, zn) float32 ...
     q_cloud_liquid_mass    (time_series_1800_1800, x, y, zn) float32 ...
     q_rain_mass            (time_series_1800_1800, x, y, zn) float32 ...
     q_cloud_liquid_number  (time_series_1800_1800, x, y, zn) float32 ...
     q_rain_number          (time_series_1800_1800, x, y, zn) float32 ...
     options_database       (number_options, kvp) |S150 ...,
 '2d': Data variables:
     surface_precip    (time_series_1800_1800, x, y) float32 ...
     vwp               (time_series_1800_1800, x, y) float32 ...
     lwp               (

In [207]:
dimensionality['3d'][0]['u'] = dimensionality['3d'][0].u.swap_dims({'x': 'xu'})

In [208]:
dimensionality['3d'][0].dims

Frozen({'time_series_1800_1800': 2, 'x': 1280, 'y': 1280, 'z': 121, 'zn': 121, 'xu': 1280, 'number_options': 664, 'kvp': 2})

In [209]:
dimensionality['3d'][0].u

<xarray.DataArray 'u' (time_series_1800_1800: 2, xu: 1280, y: 1280, zn: 121)>
[396492800 values with dtype=float32]
Coordinates:
  * time_series_1800_1800  (time_series_1800_1800) float64 1.8e+03 3.6e+03
  * zn                     (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
Dimensions without coordinates: xu, y

In [210]:
[print(list(ds.variables) == list(dimensionality['0+1d'][0].variables)) for ds in dimensionality['0+1d'][1:]]

False
True
False


[None, None, None]

In [211]:
[ds.dims for ds in dimensionality['0+1d']]

[Frozen({'time_series_300_1800': 2, 'zn': 121, 'z': 121, 'number_options': 664, 'kvp': 2}),
 Frozen({'time_series_300_1800': 2, 'zn': 121, 'z': 121, 'number_options': 664, 'kvp': 2}),
 Frozen({'time_series_300_1800': 2, 'zn': 121, 'z': 121, 'number_options': 664, 'kvp': 2}),
 Frozen({'time_series_300_1800': 2, 'zn': 121, 'z': 121, 'number_options': 664, 'kvp': 2})]

In [212]:
distinct = [list(dimensionality['0+1d'][0].variables)]
for ds in dimensionality['0+1d'][1:]:
    if list(ds.variables) not in distinct: distinct.append(list(ds.variables))

len(distinct)

2

In [213]:
outputs = {}
for d, dsets in dimensionality.items():
    if len(dsets) == 0: continue
    t = [dim for dim in dsets[0].dims if 'time' in dim][0]
    if d in outputs:
        if list(ds.variables) == list(dimensionality['0+1d'][0].variables)
        outputs[d] = xr.combine_by_coords(dsets, combine_attrs="drop_conflicts")
    else:
        outputs[d] = 


SyntaxError: expected ':' (2654974742.py, line 6)

In [214]:
list(dimensionality['0+1d'][2].data_vars)


['prefn',
 'rho',
 'rhon',
 'thref',
 'thinit',
 'u_wind_mean',
 'v_wind_mean',
 'w_wind_mean',
 'uu_mean',
 'vv_mean',
 'ww_mean',
 'www_mean',
 'wwww_mean',
 'theta_mean',
 'vapour_mmr_mean',
 'rh_mean',
 'liquid_mmr_mean',
 'rain_mmr_mean',
 'wqv_ad_mean',
 'wql_ad_mean',
 'wqr_ad_mean',
 'wqv_cn_mean',
 'wql_cn_mean',
 'wqr_cn_mean',
 'uw_mean',
 'vw_mean',
 'uv_mean',
 'wtheta_cn_mean',
 'wtheta_ad_mean',
 'wke_mean',
 'th2_mean',
 'total_cloud_fraction',
 'liquid_cloud_fraction',
 'ice_cloud_fraction',
 'pcond_mean',
 'praut_mean',
 'pracw_mean',
 'prevp_mean',
 'psedr_mean',
 'psedl_mean',
 'options_database']

In [215]:
for ds in dimensionality['0+1d']:
    print(ds.attrs)

{'title': '1d', 'created': '17/2/2022 23:6:12', 'MONC time': '7200.0', 'MONC timestep': '6360', 'Diagnostic write frequency': '3600.0', 'Previous diagnostic write at': '3600.0'}
{'title': '0d', 'created': '17/2/2022 23:6:18', 'MONC time': '7200.0', 'MONC timestep': '6360', 'Diagnostic write frequency': '3600.0', 'Previous diagnostic write at': '3600.0'}
{'title': '1d', 'created': '17/2/2022 22:55:4', 'MONC time': '3600.0', 'MONC timestep': '2620', 'Diagnostic write frequency': '3600.0', 'Previous diagnostic write at': '0.0'}
{'title': '0d', 'created': '17/2/2022 22:55:8', 'MONC time': '3600.0', 'MONC timestep': '2620', 'Diagnostic write frequency': '3600.0', 'Previous diagnostic write at': '0.0'}


In [ ]:
a = (1,5,3,6,8)
a.__iter__

<method-wrapper '__iter__' of tuple object at 0x17dee8d10>

In [ ]:
b=5
b.__iter__

AttributeError: 'int' object has no attribute '__iter__'

In [216]:
DIM_GROUPS = {(0, 1), 2, 3}

In [217]:
files =  glob('*diagnostic*.nc', root_dir=data_dir, recursive=False)
files

['d20200128_diagnostic_2d_3600.nc',
 'd20200128_diagnostic_3d_3600.nc',
 'd20200128_diagnostic_1d_7200.nc',
 'd20200128_diagnostic_0d_7200.nc',
 'd20200128_diagnostic_2d_7200.nc',
 'd20200128_diagnostic_3d_7200.nc',
 'd20200128_diagnostic_1d_3600.nc',
 'd20200128_diagnostic_0d_3600.nc']

In [ ]:
from file_functions import get_n_dims
for file in files:
    with xr.open_dataset(os.path.join(data_dir,file)) as ds:
        print(file, (get_n_dims(ds) - 1) in DIM_GROUPS)
        for group in DIM_GROUPS:
            print(group, (get_n_dims(ds) - 1) in group)
    

d20200128_diagnostic_2d_3600.nc True
(0, 1) False


TypeError: argument of type 'int' is not iterable

In [ ]:
a = 53
len(a)

TypeError: object of type 'int' has no len()

In [ ]:
DIM_GROUPS = ['0+1d', '0+1d', '2d', '3d']
dim_bins = {}
for file in files:
    with xr.open_dataset(os.path.join(data_dir,file)) as ds:
        n_dims = get_n_dims(ds) - 1
        if n_dims in dim_bins:
            dim_bins[n_dims].append(file)
        else:
            dim_bins[n_dims] = [file]
[print(d, files, DIM_GROUPS[d]) for d, files in dim_bins.items()]

2 ['d20200128_diagnostic_2d_3600.nc', 'd20200128_diagnostic_2d_7200.nc'] 2d
3 ['d20200128_diagnostic_3d_3600.nc', 'd20200128_diagnostic_3d_7200.nc'] 3d
1 ['d20200128_diagnostic_1d_7200.nc', 'd20200128_diagnostic_1d_3600.nc'] 0+1d
0 ['d20200128_diagnostic_0d_7200.nc', 'd20200128_diagnostic_0d_3600.nc'] 0+1d


[None, None, None, None]

In [ ]:
dim_bins

{2: ['d20200128_diagnostic_2d_3600.nc', 'd20200128_diagnostic_2d_7200.nc'],
 3: ['d20200128_diagnostic_3d_3600.nc', 'd20200128_diagnostic_3d_7200.nc'],
 1: ['d20200128_diagnostic_1d_7200.nc', 'd20200128_diagnostic_1d_3600.nc'],
 0: ['d20200128_diagnostic_0d_7200.nc', 'd20200128_diagnostic_0d_3600.nc']}

In [ ]:
[] + [filelist for filelist in dim_bins.values()]

[['d20200128_diagnostic_2d_3600.nc', 'd20200128_diagnostic_2d_7200.nc'],
 ['d20200128_diagnostic_3d_3600.nc', 'd20200128_diagnostic_3d_7200.nc'],
 ['d20200128_diagnostic_1d_7200.nc', 'd20200128_diagnostic_1d_3600.nc'],
 ['d20200128_diagnostic_0d_7200.nc', 'd20200128_diagnostic_0d_3600.nc']]

In [ ]:
full_list = []
for filelist in dim_bins.values():
    full_list += filelist

full_list

['d20200128_diagnostic_2d_3600.nc',
 'd20200128_diagnostic_2d_7200.nc',
 'd20200128_diagnostic_3d_3600.nc',
 'd20200128_diagnostic_3d_7200.nc',
 'd20200128_diagnostic_1d_7200.nc',
 'd20200128_diagnostic_1d_3600.nc',
 'd20200128_diagnostic_0d_7200.nc',
 'd20200128_diagnostic_0d_3600.nc']

In [ ]:
a = [1,5,3,6,8]
odd = []
i = 0
while i < len(a):
    if a[i]%2 != 0:
        odd.append(a.pop(i))
    else:
        i+=1
print(a, odd)

[6, 8] [1, 5, 3]


In [ ]:
os.path.splitext('d20200128_diagnostic_3d_7200.nc')[0]

('d20200128_diagnostic_3d_7200', '.nc')

In [ ]:
cfunits.Units(units=None, calendar='standard').formatted()

In [ ]:
print(cfunits.Units(units='s since 2020-01-25').reftime)

2020-01-25 00:00:00


In [ ]:
dim_bins[0]

['d20200128_diagnostic_0d_7200.nc', 'd20200128_diagnostic_0d_3600.nc']

In [ ]:
to_concat = [xr.open_dataset(os.path.join(data_dir, f)) for f in dim_bins[0]]
result_0d = xr.concat(to_concat,dim='time_series_300_1800', combine_attrs='drop_conflicts')
result_0d

<xarray.Dataset>
Dimensions:               (time_series_300_1800: 4, zn: 121, z: 121, number_options: 664, kvp: 2)
Coordinates:
  * time_series_300_1800  (time_series_300_1800) float64 5.4e+03 ... 3.6e+03
  * zn                    (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
  * z                     (z) float64 0.0 50.0 100.0 ... 5.9e+03 5.95e+03 6e+03
Dimensions without coordinates: number_options, kvp
Data variables: (12/23)
    VWP_mean              (time_series_300_1800) float32 23.06 23.39 22.34 22.68
    LWP_mean              (time_series_300_1800) float32 0.003599 ... 0.00279
    RWP_mean              (time_series_300_1800) float32 7.393e-07 ... 3.17e-07
    VWP_max               (time_series_300_1800) float32 25.54 26.32 22.76 24.62
    LWP_max               (time_series_300_1800) float32 0.451 0.5629 ... 0.3298
    RWP_max               (time_series_300_1800) float32 0.0007798 ... 0.0002616
    ...                    ...
    subke_max             (time_series_300_1800) float32 0.04033 ... 0.02615
    reske_min             (time_series_300_1800) float32 0.009014 ... 0.004368
    subke_min             (time_series_300_1800) float32 8.377e-05 ... 6.918e-05
    surface_precip_mean   (time_series_300_1800) float32 0.0 0.0 0.0 0.0
    surface_precip_max    (time_series_300_1800) float32 0.0 0.0 0.0 0.0
    options_database      (time_series_300_1800, number_options, kvp) |S150 b...
Attributes:
    title:                       0d
    Diagnostic write frequency:  3600.0

In [ ]:
result_0d_a = xr.combine_by_coords(to_concat,combine_attrs='drop_conflicts')
result_0d_a

<xarray.Dataset>
Dimensions:               (time_series_300_1800: 4, zn: 121, z: 121, number_options: 664, kvp: 2)
Coordinates:
  * time_series_300_1800  (time_series_300_1800) float64 1.8e+03 ... 7.2e+03
  * zn                    (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
  * z                     (z) float64 0.0 50.0 100.0 ... 5.9e+03 5.95e+03 6e+03
Dimensions without coordinates: number_options, kvp
Data variables: (12/23)
    VWP_mean              (time_series_300_1800) float32 22.34 22.68 23.06 23.39
    LWP_mean              (time_series_300_1800) float32 7.581e-07 ... 0.007011
    RWP_mean              (time_series_300_1800) float32 5.866e-15 ... 4.165e-06
    VWP_max               (time_series_300_1800) float32 22.76 24.62 25.54 26.32
    LWP_max               (time_series_300_1800) float32 0.007079 ... 0.5629
    RWP_max               (time_series_300_1800) float32 5.291e-10 ... 0.003351
    ...                    ...
    subke_max             (time_series_300_1800) float32 0.002569 ... 0.05339
    reske_min             (time_series_300_1800) float32 0.0003653 ... 0.0147
    subke_min             (time_series_300_1800) float32 8.381e-05 ... 8.53e-05
    surface_precip_mean   (time_series_300_1800) float32 0.0 0.0 0.0 0.0
    surface_precip_max    (time_series_300_1800) float32 0.0 0.0 0.0 0.0
    options_database      (time_series_300_1800, number_options, kvp) |S150 b...
Attributes:
    title:                       0d
    Diagnostic write frequency:  3600.0

So `xarray.concat` doesn't evaluate *how* the datasets should go together, so the time-points will not necessarily come out in order. `xarray.combine_by_coords` will take care of this better.

Note that any global attributes that need to be captured, but are not identical for both datasets, will need to be extracted before the merge.

In [ ]:
import numpy as np
xr.DataArray([np.nan, np.nan, 56])

<xarray.DataArray (dim_0: 3)>
array([nan, nan, 56.])
Dimensions without coordinates: dim_0

In [ ]:
from utils import type_from_str

In [ ]:
ds = dimensionality['0+1d'][0]
len(ds.time_series_300_1800.data)

2

In [ ]:
[type_from_str(ds.attrs['created'])]*len(ds.time_series_300_1800.data)

['17/2/2022 23:6:12', '17/2/2022 23:6:12']

In [ ]:

monc_timestep = type_from_str(ds.attrs['MONC timestep'])
monc_time = type_from_str(ds.attrs['MONC time'])
monc_timestep_array = xr.DataArray(
    name='monc_timestep',
    data=monc_timestep,
    coords={'time_series_300_1800': [monc_time]},
    dims=['time_series_300_1800']
)
created_array = xr.DataArray(
    data=[type_from_str(ds.attrs['created'])]*len(ds.time_series_300_1800.data),
    coords={'time_series_300_1800': ds.time_series_300_1800},
    dims='time_series_300_1800',
    name='created'
)

monc_timestep_array

<xarray.DataArray 'monc_timestep' (time_series_300_1800: 1)>
array([6360])
Coordinates:
  * time_series_300_1800  (time_series_300_1800) float64 7.2e+03

In [ ]:
created_array

<xarray.DataArray 'created' (time_series_300_1800: 2)>
array(['17/2/2022 23:6:12', '17/2/2022 23:6:12'], dtype='<U17')
Coordinates:
  * time_series_300_1800  (time_series_300_1800) float64 5.4e+03 7.2e+03

In [ ]:
ds.assign({'created': created_array, 'MONC timestep': monc_timestep_array})

<xarray.Dataset>
Dimensions:                (time_series_300_1800: 2, zn: 121, z: 121, number_options: 664, kvp: 2)
Coordinates:
  * time_series_300_1800   (time_series_300_1800) float64 5.4e+03 7.2e+03
  * zn                     (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
  * z                      (z) float64 0.0 50.0 100.0 ... 5.9e+03 5.95e+03 6e+03
Dimensions without coordinates: number_options, kvp
Data variables: (12/43)
    prefn                  (time_series_300_1800, zn) float32 ...
    rho                    (time_series_300_1800, z) float32 ...
    rhon                   (time_series_300_1800, zn) float32 ...
    thref                  (time_series_300_1800, zn) float32 ...
    thinit                 (time_series_300_1800, zn) float32 ...
    u_wind_mean            (time_series_300_1800, zn) float32 ...
    ...                     ...
    prevp_mean             (time_series_300_1800, zn) float32 ...
    psedr_mean             (time_series_300_1800, zn) float32 ...
    psedl_mean             (time_series_300_1800, zn) float32 ...
    options_database       (number_options, kvp) |S150 ...
    created                (time_series_300_1800) <U17 '17/2/2022 23:6:12' '1...
    MONC timestep          (time_series_300_1800) float64 nan 6.36e+03
Attributes:
    title:                         1d
    created:                       17/2/2022 23:6:12
    MONC time:                     7200.0
    MONC timestep:                 6360
    Diagnostic write frequency:    3600.0
    Previous diagnostic write at:  3600.0

In [ ]:
a = {'breakfast': 'spam'}
a.update({'beverage':'tea'})
a

{'breakfast': 'spam', 'beverage': 'tea'}

In [ ]:
a.update({'breakfast':'spam, eggs, bacon, beans and spam'})
a

{'breakfast': 'spam, eggs, bacon, beans and spam', 'beverage': 'tea'}

In [ ]:
ds

<xarray.Dataset>
Dimensions:                (time_series_300_1800: 2, zn: 121, z: 121, number_options: 664, kvp: 2)
Coordinates:
  * time_series_300_1800   (time_series_300_1800) float64 5.4e+03 7.2e+03
  * zn                     (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
  * z                      (z) float64 0.0 50.0 100.0 ... 5.9e+03 5.95e+03 6e+03
Dimensions without coordinates: number_options, kvp
Data variables: (12/41)
    prefn                  (time_series_300_1800, zn) float32 ...
    rho                    (time_series_300_1800, z) float32 ...
    rhon                   (time_series_300_1800, zn) float32 ...
    thref                  (time_series_300_1800, zn) float32 ...
    thinit                 (time_series_300_1800, zn) float32 ...
    u_wind_mean            (time_series_300_1800, zn) float32 ...
    ...                     ...
    praut_mean             (time_series_300_1800, zn) float32 ...
    pracw_mean             (time_series_300_1800, zn) float32 ...
    prevp_mean             (time_series_300_1800, zn) float32 ...
    psedr_mean             (time_series_300_1800, zn) float32 ...
    psedl_mean             (time_series_300_1800, zn) float32 ...
    options_database       (number_options, kvp) |S150 ...
Attributes:
    title:                         1d
    created:                       17/2/2022 23:6:12
    MONC time:                     7200.0
    MONC timestep:                 6360
    Diagnostic write frequency:    3600.0
    Previous diagnostic write at:  3600.0

In [ ]:
type(ds.data_vars)

xarray.core.dataset.DataVariables

In [ ]:
type(ds.coords)

xarray.core.coordinates.DatasetCoordinates

In [ ]:
type(ds.variables)

xarray.core.utils.Frozen

In [ ]:
dvs = []
{k: [a for a in v] for k, v in ds.data_vars.items()}

{'prefn': [<xarray.DataArray 'prefn' (zn: 121)>
  array([101888.07 , 101309.49 , 100733.26 , 100159.37 ,  99587.82 ,  99018.6  ,
          98451.72 ,  97887.16 ,  97324.91 ,  96764.97 ,  96207.336,  95652.   ,
          95098.96 ,  94548.21 ,  93999.74 ,  93453.555,  92909.63 ,  92367.98 ,
          91828.586,  91291.44 ,  90756.55 ,  90223.9  ,  89693.484,  89165.3  ,
          88639.34 ,  88115.61 ,  87594.086,  87074.77 ,  86557.66 ,  86042.75 ,
          85530.02 ,  85019.49 ,  84511.14 ,  84004.96 ,  83500.95 ,  82999.1  ,
          82499.414,  82001.88 ,  81506.5  ,  81013.25 ,  80522.14 ,  80033.164,
          79546.31 ,  79061.58 ,  78578.95 ,  78098.445,  77620.03 ,  77143.72 ,
          76669.5  ,  76197.37 ,  75727.31 ,  75259.336,  74793.42 ,  74329.58 ,
          73867.8  ,  73408.06 ,  72950.375,  72494.734,  72041.125,  71589.555,
          71140.   ,  70692.48 ,  70246.96 ,  69803.45 ,  69361.95 ,  68922.45 ,
          68484.94 ,  68049.42 ,  67615.875,  67184.31 ,  667

In [ ]:
dvs

[<xarray.DataArray ()>
 array('prefn', dtype='<U5'),
 <xarray.DataArray ()>
 array('rho', dtype='<U3'),
 <xarray.DataArray ()>
 array('rhon', dtype='<U4'),
 <xarray.DataArray ()>
 array('thref', dtype='<U5'),
 <xarray.DataArray ()>
 array('thinit', dtype='<U6'),
 <xarray.DataArray ()>
 array('u_wind_mean', dtype='<U11'),
 <xarray.DataArray ()>
 array('v_wind_mean', dtype='<U11'),
 <xarray.DataArray ()>
 array('w_wind_mean', dtype='<U11'),
 <xarray.DataArray ()>
 array('uu_mean', dtype='<U7'),
 <xarray.DataArray ()>
 array('vv_mean', dtype='<U7'),
 <xarray.DataArray ()>
 array('ww_mean', dtype='<U7'),
 <xarray.DataArray ()>
 array('www_mean', dtype='<U8'),
 <xarray.DataArray ()>
 array('wwww_mean', dtype='<U9'),
 <xarray.DataArray ()>
 array('theta_mean', dtype='<U10'),
 <xarray.DataArray ()>
 array('vapour_mmr_mean', dtype='<U15'),
 <xarray.DataArray ()>
 array('rh_mean', dtype='<U7'),
 <xarray.DataArray ()>
 array('liquid_mmr_mean', dtype='<U15'),
 <xarray.DataArray ()>
 array('rain_m

In [ ]:
dvs[0]

<xarray.DataArray ()>
array('prefn', dtype='<U5')

In [ ]:
'fastpath' in [attr for attr in dir(list(ds.data_vars.values())[0])]

False

In [ ]:
a = 'fizzgig'
list(a)

['f', 'i', 'z', 'z', 'g', 'i', 'g']

In [ ]:
def arg_check(*args):
    print(args)
    print(len(args))
    print(len(list(args)))
    print(list(args))


In [ ]:
arg_check('spam')

('spam',)
1
1
['spam']


In [ ]:
filepaths = [os.path.join(data_dir, f) for f in glob('*.nc', root_dir=data_dir, recursive=False)]

In [ ]:
filenames = [os.path.splitext(os.path.basename(path))[0] for path in filepaths]

In [ ]:
stem = filenames[0]
for f in filenames[1:]:
    match = SequenceMatcher(a=stem, b=f).find_longest_match()
    stem = stem[match.a: match.a + match.size]
stem

'20200128_'

In [ ]:
filenames = [os.path.splitext(os.path.basename(path))[0] for path in (stem, 'test_data/d20200128_diagnostic_2d_7200.nc')]
filenames

['20200128_', 'd20200128_diagnostic_2d_7200']

In [ ]:
stem = filenames[0]
for f in filenames[1:]:
    match = SequenceMatcher(a=stem, b=f).find_longest_match()
    stem = stem[match.a: match.a + match.size]
stem

'20200128_'

In [ ]:
cfunits.Units('kg/kg').formatted()

'1'

In [ ]:
[a for a in None]

TypeError: 'NoneType' object is not iterable

In [ ]:
dimensionality

{'3d': [<xarray.Dataset>
  Dimensions:                (time_series_1800_1800: 2, x: 1280, y: 1280, z: 121, zn: 121, xu: 1280, number_options: 664, kvp: 2)
  Coordinates:
    * time_series_1800_1800  (time_series_1800_1800) float64 1.8e+03 3.6e+03
    * zn                     (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
    * z                      (z) float64 0.0 50.0 100.0 ... 5.9e+03 5.95e+03 6e+03
  Dimensions without coordinates: x, y, xu, number_options, kvp
  Data variables:
      w                      (time_series_1800_1800, x, y, z) float32 ...
      v                      (time_series_1800_1800, x, y, zn) float32 ...
      u                      (time_series_1800_1800, xu, y, zn) float32 ...
      th                     (time_series_1800_1800, x, y, zn) float32 ...
      p                      (time_series_1800_1800, x, y, zn) float32 ...
      q_vapour               (time_series_1800_1800, x, y, zn) float32 ...
      q_cloud_liquid_mass    (time_series_1800_1800, x, 

In [ ]:
dimensionality.keys()


dict_keys(['3d', '2d', '0+1d'])

In [ ]:
to_merge_dims = [ds for ds in dimensionality['0+1d'] if ds.attrs['MONC time'] == '7200.0']
to_merge_dims

[<xarray.Dataset>
 Dimensions:                (time_series_300_1800: 2, zn: 121, z: 121, number_options: 664, kvp: 2)
 Coordinates:
   * time_series_300_1800   (time_series_300_1800) float64 5.4e+03 7.2e+03
   * zn                     (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
   * z                      (z) float64 0.0 50.0 100.0 ... 5.9e+03 5.95e+03 6e+03
 Dimensions without coordinates: number_options, kvp
 Data variables: (12/41)
     prefn                  (time_series_300_1800, zn) float32 1.019e+05 ... 4...
     rho                    (time_series_300_1800, z) float32 1.18 ... 0.6835
     rhon                   (time_series_300_1800, zn) float32 1.182 ... 0.687
     thref                  (time_series_300_1800, zn) float32 298.7 ... 298.7
     thinit                 (time_series_300_1800, zn) float32 0.0 0.0 ... 0.0
     u_wind_mean            (time_series_300_1800, zn) float32 6.608 ... 5.46
     ...                     ...
     praut_mean             (time_series_300

In [ ]:
to_merge_dims[0].update(to_merge_dims[1])  # Merges to_merge_dims[1] INTO to_merge_dims[0]

<xarray.Dataset>
Dimensions:                (time_series_300_1800: 2, zn: 121, z: 121, number_options: 664, kvp: 2)
Coordinates:
  * time_series_300_1800   (time_series_300_1800) float64 5.4e+03 7.2e+03
  * zn                     (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
  * z                      (z) float64 0.0 50.0 100.0 ... 5.9e+03 5.95e+03 6e+03
Dimensions without coordinates: number_options, kvp
Data variables: (12/63)
    prefn                  (time_series_300_1800, zn) float32 1.019e+05 ... 4...
    rho                    (time_series_300_1800, z) float32 1.18 ... 0.6835
    rhon                   (time_series_300_1800, zn) float32 1.182 ... 0.687
    thref                  (time_series_300_1800, zn) float32 298.7 ... 298.7
    thinit                 (time_series_300_1800, zn) float32 0.0 0.0 ... 0.0
    u_wind_mean            (time_series_300_1800, zn) float32 6.608 ... 5.46
    ...                     ...
    reske_max              (time_series_300_1800) float32 ...
    subke_max              (time_series_300_1800) float32 ...
    reske_min              (time_series_300_1800) float32 ...
    subke_min              (time_series_300_1800) float32 ...
    surface_precip_mean    (time_series_300_1800) float32 ...
    surface_precip_max     (time_series_300_1800) float32 ...
Attributes:
    title:                         1d
    created:                       17/2/2022 23:6:12
    MONC time:                     7200.0
    MONC timestep:                 6360
    Diagnostic write frequency:    3600.0
    Previous diagnostic write at:  3600.0

In [ ]:
to_merge_dims = [ds for ds in dimensionality['0+1d'] if ds.attrs['MONC time'] == '3600.0']

In [ ]:
to_merge_dims[0].merge(other=to_merge_dims[1], join='exact', combine_attrs='drop_conflicts')  # Creates a new dataset

<xarray.Dataset>
Dimensions:                (time_series_300_1800: 2, zn: 121, z: 121, number_options: 664, kvp: 2)
Coordinates:
  * time_series_300_1800   (time_series_300_1800) float64 1.8e+03 3.6e+03
  * zn                     (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
  * z                      (z) float64 0.0 50.0 100.0 ... 5.9e+03 5.95e+03 6e+03
Dimensions without coordinates: number_options, kvp
Data variables: (12/63)
    prefn                  (time_series_300_1800, zn) float32 ...
    rho                    (time_series_300_1800, z) float32 ...
    rhon                   (time_series_300_1800, zn) float32 ...
    thref                  (time_series_300_1800, zn) float32 ...
    thinit                 (time_series_300_1800, zn) float32 ...
    u_wind_mean            (time_series_300_1800, zn) float32 ...
    ...                     ...
    reske_max              (time_series_300_1800) float32 ...
    subke_max              (time_series_300_1800) float32 ...
    reske_min              (time_series_300_1800) float32 ...
    subke_min              (time_series_300_1800) float32 ...
    surface_precip_mean    (time_series_300_1800) float32 ...
    surface_precip_max     (time_series_300_1800) float32 ...
Attributes:
    MONC time:                     3600.0
    MONC timestep:                 2620
    Diagnostic write frequency:    3600.0
    Previous diagnostic write at:  0.0

`xarray.Dataset.merge()` does have options to determine how to resolve conflicts in coordinate points and attributes, but the above version is best for this application.

In [ ]:
def merge_test(*args: str):
    
    to_merge = args
    # print(len(to_merge), to_merge)

    if len(to_merge) == 1:
        return to_merge[0]

    if len(to_merge) == 2:
        # print('returning', ', '.join(to_merge))
        return ', '.join(to_merge)
    
    else:
        # call recursively until only 2 args to process
        # print('calling merge test on', to_merge[1:])
        # merge_from_1 = merge_test(*to_merge[1:])
        # print(merge_from_1)
        # return merge_test(to_merge[0], merge_from_1)
        return merge_test(to_merge[0], merge_test(*to_merge[1:]))

data = ['one string']
print('Testing with single argument:', merge_test(*data))

data = ['one string', 'another string']
print('Testing with 2 arguments:', merge_test(*data))

data = ['string one', 'string two', 'string 3', '4', 'string one million']
print('Testing with 5 arguments:', merge_test(*data))

Testing with single argument: one string
Testing with 2 arguments: one string, another string
Testing with 5 arguments: string one, string two, string 3, 4, string one million


In [ ]:
dimensionality['0+1d'][1]

<xarray.Dataset>
Dimensions:               (time_series_300_1800: 2, zn: 121, z: 121, number_options: 664, kvp: 2)
Coordinates:
  * time_series_300_1800  (time_series_300_1800) float64 5.4e+03 7.2e+03
  * zn                    (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
  * z                     (z) float64 0.0 50.0 100.0 ... 5.9e+03 5.95e+03 6e+03
Dimensions without coordinates: number_options, kvp
Data variables: (12/23)
    VWP_mean              (time_series_300_1800) float32 ...
    LWP_mean              (time_series_300_1800) float32 ...
    RWP_mean              (time_series_300_1800) float32 ...
    VWP_max               (time_series_300_1800) float32 ...
    LWP_max               (time_series_300_1800) float32 ...
    RWP_max               (time_series_300_1800) float32 ...
    ...                    ...
    subke_max             (time_series_300_1800) float32 ...
    reske_min             (time_series_300_1800) float32 ...
    subke_min             (time_series_300_1800) float32 ...
    surface_precip_mean   (time_series_300_1800) float32 ...
    surface_precip_max    (time_series_300_1800) float32 ...
    options_database      (number_options, kvp) |S150 ...
Attributes:
    title:                         0d
    created:                       17/2/2022 23:6:18
    MONC time:                     7200.0
    MONC timestep:                 6360
    Diagnostic write frequency:    3600.0
    Previous diagnostic write at:  3600.0

In [230]:
class TestArgs:
    def __init__(self) -> None:
        self.a = 'A'
    
    def get(self, attr):
        return self.__getattribute__(attr)
    
thing = TestArgs()
print(thing.get('a'))
print(thing.get('b'))

A


AttributeError: 'TestArgs' object has no attribute 'b'

In [231]:
{a: TestArgs().get('a') for a in ['moo', 'squares', 'milk']}

{'moo': 'A', 'squares': 'A', 'milk': 'A'}

In [232]:
b = {a: TestArgs() for a in ['moo', 'squares', 'milk']}
[x.get('a') for x in b.values()]

['A', 'A', 'A']

In [ ]:
dir(thing)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'a',
 'get']

In [ ]:
{k:v for k,v in options_db.items() if 'dephy' in k.lower()}

{'dephy_forcings_enabled': 'true',
 'dephy_file': 'eurec4a_20200128_first.dephy_inversion_nudging.nc',
 'dephy_forcing_enabled': 'false'}

In [ ]:
def tf(a):
    if a:
        print(f'{a} considered true')
    else:
        print(f'{a} considered false')

tf('')
tf(None)
tf('thing')
tf(42)

 considered false
None considered false
thing considered true
42 considered true


In [ ]:
latlonz0 = ["longitude", "latitude", "z0"]
{k:v for k,v in options_db.items() if k.lower() in latlonz0}

{'longitude': '-90.0', 'z0': '0.00020', 'latitude': '0.0'}

In [ ]:
a = []
[a.append(i) for i in range(20) if i%2 == 0]
a

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

In [ ]:
DIM_GROUPS = {
        0: '0+1d', 
        1: '0+1d', 
        2: '2d', 
        3: '3d'
    }

{g:list(DIM_GROUPS.values()).count(g) for g in set(DIM_GROUPS.values())}

{'2d': 1, '0+1d': 2, '3d': 1}

In [ ]:
[g for g in set(DIM_GROUPS.values()) if list(DIM_GROUPS.values()).count(g) > 1]

['0+1d']

In [ ]:
for k, v in dimensionality.items():
    for ds in v:
        ds.close()

In [ ]:
nc1d_1.close()

In [ ]:
a = 1
b = {a}
b

{1}

In [219]:
len(dimensionality['2d'])

2

In [224]:
time_vars = [d for d in dimensionality['2d'][0].dims if 'time' in d]
time_var = time_vars[0]
time_var

'time_series_1800_1800'

In [225]:
xr.combine_by_coords(
            [ds for ds in dimensionality['2d']],
            combine_attrs='drop_conflicts',
            join='exact',
            coords=[time_var],
            data_vars='minimal'
        )

<xarray.Dataset>
Dimensions:                (time_series_1800_1800: 4, x: 1280, y: 1280, zn: 121, z: 121, number_options: 664, kvp: 2)
Coordinates:
  * time_series_1800_1800  (time_series_1800_1800) float64 1.8e+03 ... 7.2e+03
  * zn                     (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
  * z                      (z) float64 0.0 50.0 100.0 ... 5.9e+03 5.95e+03 6e+03
Dimensions without coordinates: x, y, number_options, kvp
Data variables:
    surface_precip         (time_series_1800_1800, x, y) float32 0.0 0.0 ... 0.0
    vwp                    (time_series_1800_1800, x, y) float32 22.38 ... 23.67
    lwp                    (time_series_1800_1800, x, y) float32 0.0 0.0 ... 0.0
    rwp                    (time_series_1800_1800, x, y) float32 0.0 0.0 ... 0.0
    senhf                  (time_series_1800_1800, x, y) float32 10.5 ... 11.95
    lathf                  (time_series_1800_1800, x, y) float32 172.5 ... 183.1
    reske                  (time_series_1800_1800, x, y) float32 0.01209 ... ...
    subke                  (time_series_1800_1800, x, y) float32 0.0007217 .....
    cltop                  (time_series_1800_1800, x, y) float32 0.0 0.0 ... 0.0
    clbas                  (time_series_1800_1800, x, y) float32 0.0 0.0 ... 0.0
    options_database       (number_options, kvp) |S150 b'rad_interval' ... b'...
Attributes:
    title:                       2d
    Diagnostic write frequency:  3600.0

In [229]:
a = [2,5,2]
q = {"proc": a}
isinstance(q["proc"], int)

False

In [235]:
DIM_GROUPS = {
        0: '0+1d', 
        1: '0+1d', 
        2: '2d', 
        3: '3d'
    }
for g in ['0+1d', '2d', '3d']:
    print([group for group in DIM_GROUPS.values() if group == g])

['0+1d', '0+1d']
['2d']
['3d']


In [236]:
a[-2:]

[5, 2]

In [239]:
list(dimensionality['3d'][0].variables.keys())

['time_series_1800_1800',
 'w',
 'v',
 'u',
 'th',
 'p',
 'q_vapour',
 'q_cloud_liquid_mass',
 'q_rain_mass',
 'q_cloud_liquid_number',
 'q_rain_number',
 'zn',
 'z',
 'options_database']

In [17]:
a = []
if a:
    print('I count empty list as True')
else:
    print('Empty lists equate to False')

Empty lists equate to False


In [26]:
a = {'face': 'Bert N'}
b = a['axis'] if 'axis' in a else 'Nowt'
b

'Nowt'

In [42]:
nc1d_1['prefn']

<xarray.DataArray 'prefn' (time_series_300_1800: 2, zn: 121)>
array([[101888.07 , 101309.49 , 100733.26 , ...,  48347.65 ,  48008.12 ,
         47670.297],
       [101888.07 , 101309.49 , 100733.26 , ...,  48347.65 ,  48008.12 ,
         47670.297]], dtype=float32)
Coordinates:
  * time_series_300_1800  (time_series_300_1800) float64 1.8e+03 3.6e+03
  * zn                    (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
Attributes:
    units:    hPa

In [43]:
nc1d_1['prefn'].dims

('time_series_300_1800', 'zn')

In [44]:
update = {'x': 'xu'}
set(update.keys()).issubset(set(nc1d_1['prefn'].dims))

False

In [45]:
update = {'zn': 'z'}
set(update.keys()).issubset(set(nc1d_1['prefn'].dims))

True

In [60]:
nc1d_1['prefn']

<xarray.DataArray 'prefn' (time_series_300_1800: 2, zn: 121)>
array([[101888.07 , 101309.49 , 100733.26 , ...,  48347.65 ,  48008.12 ,
         47670.297],
       [101888.07 , 101309.49 , 100733.26 , ...,  48347.65 ,  48008.12 ,
         47670.297]], dtype=float32)
Coordinates:
  * time_series_300_1800  (time_series_300_1800) float64 1.8e+03 3.6e+03
  * zn                    (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
Attributes:
    units:    hPa

In [61]:
'prefn' in nc1d_1.variables

True

In [63]:
nc1d_1['prefn'].coords

Coordinates:
  * time_series_300_1800  (time_series_300_1800) float64 1.8e+03 3.6e+03
  * zn                    (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03

In [64]:
nc1d_1['zn'].coords

Coordinates:
  * zn       (zn) float64 -25.0 25.0 75.0 ... 5.875e+03 5.925e+03 5.975e+03

In [65]:
nc1d_1['z'].coords

Coordinates:
  * z        (z) float64 0.0 50.0 100.0 150.0 ... 5.9e+03 5.95e+03 6e+03

In [66]:
nc1d_1['time_series_300_1800'].coords

Coordinates:
  * time_series_300_1800  (time_series_300_1800) float64 1.8e+03 3.6e+03

In [68]:
nc1d_1['time_series_300_1800'].name in nc1d_1['time_series_300_1800'].coords

True

In [50]:
new_var = list(nc1d_1.variables.values())[1].copy(deep=True)

In [69]:
{'x','y','z','time'}.intersection({'time_series', 'x-coordinate', 'zn', 'y-coordinate'})

set()

In [70]:
'time' in {'time series', 'spam'}

False

In [52]:
new_var in list(nc1d_1.variables.values())

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [75]:
current = cfunits.Units(nc1d_1['prefn'].units)
new_u = cfunits.Units('Pa')

In [76]:
current.equals(new_u)

False

In [77]:
current.equivalent(new_u)

True

In [85]:
nc1d_1['prefn'].data

array([[101888.07 , 101309.49 , 100733.26 , 100159.37 ,  99587.82 ,
         99018.6  ,  98451.72 ,  97887.16 ,  97324.91 ,  96764.97 ,
         96207.336,  95652.   ,  95098.96 ,  94548.21 ,  93999.74 ,
         93453.555,  92909.63 ,  92367.98 ,  91828.586,  91291.44 ,
         90756.55 ,  90223.9  ,  89693.484,  89165.3  ,  88639.34 ,
         88115.61 ,  87594.086,  87074.77 ,  86557.66 ,  86042.75 ,
         85530.02 ,  85019.49 ,  84511.14 ,  84004.96 ,  83500.95 ,
         82999.1  ,  82499.414,  82001.88 ,  81506.5  ,  81013.25 ,
         80522.14 ,  80033.164,  79546.31 ,  79061.58 ,  78578.95 ,
         78098.445,  77620.03 ,  77143.72 ,  76669.5  ,  76197.37 ,
         75727.31 ,  75259.336,  74793.42 ,  74329.58 ,  73867.8  ,
         73408.06 ,  72950.375,  72494.734,  72041.125,  71589.555,
         71140.   ,  70692.48 ,  70246.96 ,  69803.45 ,  69361.95 ,
         68922.45 ,  68484.94 ,  68049.42 ,  67615.875,  67184.31 ,
         66754.73 ,  66327.1  ,  65901.43 ,  654

In [87]:
cfunits.Units.conform(x=nc1d_1['prefn'].data, from_units=current, to_units=new_u, inplace=True)

array([[10188807. , 10130949. , 10073326. , 10015937. ,  9958782. ,
         9901860. ,  9845172. ,  9788716. ,  9732491. ,  9676497. ,
         9620734. ,  9565200. ,  9509896. ,  9454821. ,  9399974. ,
         9345355. ,  9290963. ,  9236798. ,  9182859. ,  9129144. ,
         9075655. ,  9022390. ,  8969348. ,  8916530. ,  8863934. ,
         8811561. ,  8759409. ,  8707477. ,  8655766. ,  8604275. ,
         8553002. ,  8501949. ,  8451114. ,  8400496. ,  8350095.5,
         8299910. ,  8249941.5,  8200188.5,  8150650. ,  8101325. ,
         8052214. ,  8003316.5,  7954631. ,  7906158. ,  7857895.5,
         7809844.5,  7762003. ,  7714372. ,  7666950. ,  7619736.5,
         7572731. ,  7525933.5,  7479342. ,  7432958. ,  7386779.5,
         7340806. ,  7295037.5,  7249473.5,  7204112.5,  7158955.5,
         7114000. ,  7069247.5,  7024696. ,  6980345.5,  6936195.5,
         6892245.5,  6848494. ,  6804942. ,  6761587.5,  6718431. ,
         6675472.5,  6632710. ,  6590143. ,  654

In [88]:
nc1d_1['prefn'].data

array([[10188807. , 10130949. , 10073326. , 10015937. ,  9958782. ,
         9901860. ,  9845172. ,  9788716. ,  9732491. ,  9676497. ,
         9620734. ,  9565200. ,  9509896. ,  9454821. ,  9399974. ,
         9345355. ,  9290963. ,  9236798. ,  9182859. ,  9129144. ,
         9075655. ,  9022390. ,  8969348. ,  8916530. ,  8863934. ,
         8811561. ,  8759409. ,  8707477. ,  8655766. ,  8604275. ,
         8553002. ,  8501949. ,  8451114. ,  8400496. ,  8350095.5,
         8299910. ,  8249941.5,  8200188.5,  8150650. ,  8101325. ,
         8052214. ,  8003316.5,  7954631. ,  7906158. ,  7857895.5,
         7809844.5,  7762003. ,  7714372. ,  7666950. ,  7619736.5,
         7572731. ,  7525933.5,  7479342. ,  7432958. ,  7386779.5,
         7340806. ,  7295037.5,  7249473.5,  7204112.5,  7158955.5,
         7114000. ,  7069247.5,  7024696. ,  6980345.5,  6936195.5,
         6892245.5,  6848494. ,  6804942. ,  6761587.5,  6718431. ,
         6675472.5,  6632710. ,  6590143. ,  654

In [90]:
nc1d_1['prefn'].attrs['units'] = new_u.formatted()

In [91]:
nc1d_1['prefn']

<xarray.DataArray 'prefn' (time_series_300_1800: 2, zn: 121)>
array([[10188807. , 10130949. , 10073326. , ...,  4834765. ,  4800812. ,
         4767029.5],
       [10188807. , 10130949. , 10073326. , ...,  4834765. ,  4800812. ,
         4767029.5]], dtype=float32)
Coordinates:
  * time_series_300_1800  (time_series_300_1800) float64 1.8e+03 3.6e+03
  * zn                    (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
Attributes:
    units:    Pa

In [97]:
nc1d_1.time_series_300_1800

<xarray.DataArray 'time_series_300_1800' (time_series_300_1800: 2)>
array([1800., 3600.])
Coordinates:
  * time_series_300_1800  (time_series_300_1800) float64 1.8e+03 3.6e+03

In [113]:
variables = {}

In [114]:
variables.update({v: '0+1d' for v in nc1d_1.variables if v not in variables})
variables


{'time_series_300_1800': '0+1d',
 'prefn': '0+1d',
 'rho': '0+1d',
 'rhon': '0+1d',
 'thref': '0+1d',
 'thinit': '0+1d',
 'u_wind_mean': '0+1d',
 'v_wind_mean': '0+1d',
 'w_wind_mean': '0+1d',
 'uu_mean': '0+1d',
 'vv_mean': '0+1d',
 'ww_mean': '0+1d',
 'www_mean': '0+1d',
 'wwww_mean': '0+1d',
 'theta_mean': '0+1d',
 'vapour_mmr_mean': '0+1d',
 'rh_mean': '0+1d',
 'liquid_mmr_mean': '0+1d',
 'rain_mmr_mean': '0+1d',
 'wqv_ad_mean': '0+1d',
 'wql_ad_mean': '0+1d',
 'wqr_ad_mean': '0+1d',
 'wqv_cn_mean': '0+1d',
 'wql_cn_mean': '0+1d',
 'wqr_cn_mean': '0+1d',
 'uw_mean': '0+1d',
 'vw_mean': '0+1d',
 'uv_mean': '0+1d',
 'wtheta_cn_mean': '0+1d',
 'wtheta_ad_mean': '0+1d',
 'wke_mean': '0+1d',
 'th2_mean': '0+1d',
 'total_cloud_fraction': '0+1d',
 'liquid_cloud_fraction': '0+1d',
 'ice_cloud_fraction': '0+1d',
 'pcond_mean': '0+1d',
 'praut_mean': '0+1d',
 'pracw_mean': '0+1d',
 'prevp_mean': '0+1d',
 'psedr_mean': '0+1d',
 'psedl_mean': '0+1d',
 'zn': '0+1d',
 'z': '0+1d',
 'options_data

In [115]:
len(variables)

44

In [105]:
nc0d_1 = xr.open_dataset(os.path.join(data_dir, 'd20200128_diagnostic_0d_3600.nc'))

In [116]:
variables.update({v: '0+1d' for v in nc0d_1.variables if v not in variables})
variables

{'time_series_300_1800': '0+1d',
 'prefn': '0+1d',
 'rho': '0+1d',
 'rhon': '0+1d',
 'thref': '0+1d',
 'thinit': '0+1d',
 'u_wind_mean': '0+1d',
 'v_wind_mean': '0+1d',
 'w_wind_mean': '0+1d',
 'uu_mean': '0+1d',
 'vv_mean': '0+1d',
 'ww_mean': '0+1d',
 'www_mean': '0+1d',
 'wwww_mean': '0+1d',
 'theta_mean': '0+1d',
 'vapour_mmr_mean': '0+1d',
 'rh_mean': '0+1d',
 'liquid_mmr_mean': '0+1d',
 'rain_mmr_mean': '0+1d',
 'wqv_ad_mean': '0+1d',
 'wql_ad_mean': '0+1d',
 'wqr_ad_mean': '0+1d',
 'wqv_cn_mean': '0+1d',
 'wql_cn_mean': '0+1d',
 'wqr_cn_mean': '0+1d',
 'uw_mean': '0+1d',
 'vw_mean': '0+1d',
 'uv_mean': '0+1d',
 'wtheta_cn_mean': '0+1d',
 'wtheta_ad_mean': '0+1d',
 'wke_mean': '0+1d',
 'th2_mean': '0+1d',
 'total_cloud_fraction': '0+1d',
 'liquid_cloud_fraction': '0+1d',
 'ice_cloud_fraction': '0+1d',
 'pcond_mean': '0+1d',
 'praut_mean': '0+1d',
 'pracw_mean': '0+1d',
 'prevp_mean': '0+1d',
 'psedr_mean': '0+1d',
 'psedl_mean': '0+1d',
 'zn': '0+1d',
 'z': '0+1d',
 'options_data

In [117]:
len(variables)

66

In [1]:
a = {1,4,8}
b = {2}
a.union(b)

{1, 2, 4, 8}

In [2]:
datetime.now

NameError: name 'datetime' is not defined

In [64]:
filename = 'd20200128_diagnostic_3d_3600.nc'
filepath = os.path.join('../test_data/', filename)
nc3d = xr.open_dataset(filepath)
nc3d

<xarray.Dataset>
Dimensions:                (time_series_1800_1800: 2, x: 1280, y: 1280, z: 121, zn: 121, number_options: 664, kvp: 2)
Coordinates:
  * time_series_1800_1800  (time_series_1800_1800) float64 1.8e+03 3.6e+03
  * zn                     (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
  * z                      (z) float64 0.0 50.0 100.0 ... 5.9e+03 5.95e+03 6e+03
Dimensions without coordinates: x, y, number_options, kvp
Data variables:
    w                      (time_series_1800_1800, x, y, z) float32 ...
    v                      (time_series_1800_1800, x, y, zn) float32 ...
    u                      (time_series_1800_1800, x, y, zn) float32 ...
    th                     (time_series_1800_1800, x, y, zn) float32 ...
    p                      (time_series_1800_1800, x, y, zn) float32 ...
    q_vapour               (time_series_1800_1800, x, y, zn) float32 ...
    q_cloud_liquid_mass    (time_series_1800_1800, x, y, zn) float32 ...
    q_rain_mass            (time_series_1800_1800, x, y, zn) float32 ...
    q_cloud_liquid_number  (time_series_1800_1800, x, y, zn) float32 ...
    q_rain_number          (time_series_1800_1800, x, y, zn) float32 ...
    options_database       (number_options, kvp) |S150 ...
Attributes:
    title:                         3d
    created:                       17/2/2022 22:51:17
    MONC time:                     3600.0
    MONC timestep:                 2620
    Diagnostic write frequency:    3600.0
    Previous diagnostic write at:  0.0

In [7]:
nc3d.title

'3d'

In [65]:
filename = 'd20200128_diagnostic_1d_3600.nc'
filepath = os.path.join('../test_data/', filename)
nc1d = xr.open_dataset(filepath)
nc1d

<xarray.Dataset>
Dimensions:                (time_series_300_1800: 2, zn: 121, z: 121, number_options: 664, kvp: 2)
Coordinates:
  * time_series_300_1800   (time_series_300_1800) float64 1.8e+03 3.6e+03
  * zn                     (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
  * z                      (z) float64 0.0 50.0 100.0 ... 5.9e+03 5.95e+03 6e+03
Dimensions without coordinates: number_options, kvp
Data variables: (12/41)
    prefn                  (time_series_300_1800, zn) float32 ...
    rho                    (time_series_300_1800, z) float32 ...
    rhon                   (time_series_300_1800, zn) float32 ...
    thref                  (time_series_300_1800, zn) float32 ...
    thinit                 (time_series_300_1800, zn) float32 ...
    u_wind_mean            (time_series_300_1800, zn) float32 ...
    ...                     ...
    praut_mean             (time_series_300_1800, zn) float32 ...
    pracw_mean             (time_series_300_1800, zn) float32 ...
    prevp_mean             (time_series_300_1800, zn) float32 ...
    psedr_mean             (time_series_300_1800, zn) float32 ...
    psedl_mean             (time_series_300_1800, zn) float32 ...
    options_database       (number_options, kvp) |S150 ...
Attributes:
    title:                         1d
    created:                       17/2/2022 22:55:4
    MONC time:                     3600.0
    MONC timestep:                 2620
    Diagnostic write frequency:    3600.0
    Previous diagnostic write at:  0.0

In [9]:
nc3d.p

<xarray.DataArray 'p' (time_series_1800_1800: 2, x: 1280, y: 1280, zn: 121)>
[396492800 values with dtype=float32]
Coordinates:
  * time_series_1800_1800  (time_series_1800_1800) float64 1.8e+03 3.6e+03
  * zn                     (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
Dimensions without coordinates: x, y

In [11]:
nc3d.p.shape

(2, 1280, 1280, 121)

In [10]:
nc1d.prefn

<xarray.DataArray 'prefn' (time_series_300_1800: 2, zn: 121)>
array([[101888.07 , 101309.49 , 100733.26 , ...,  48347.65 ,  48008.12 ,
         47670.297],
       [101888.07 , 101309.49 , 100733.26 , ...,  48347.65 ,  48008.12 ,
         47670.297]], dtype=float32)
Coordinates:
  * time_series_300_1800  (time_series_300_1800) float64 1.8e+03 3.6e+03
  * zn                    (zn) float64 -25.0 25.0 75.0 ... 5.925e+03 5.975e+03
Attributes:
    units:    hPa

In [12]:
nc1d.prefn.shape

(2, 121)

In [14]:
nc3d = nc3d.rename({'time_series_1800_1800': 'time'})
nc1d = nc1d.rename({'time_series_300_1800': 'time'})

In [16]:
p_abs = nc3d.p + nc1d.prefn

In [47]:
nc3d.p.sel(time=slice(1800., 3600.), x=1, y=1, zn=25.).data

array([ 0.3078205 , -0.14059114], dtype=float32)

In [44]:
ref_p = nc1d.prefn.sel(time=slice(1800., 3600.), zn=25.).data
ref_p

array([101309.49, 101309.49], dtype=float32)

In [46]:
p_abs.sel(time=slice(1800., 3600.), x=1, y=1, zn=25.).data

array([101309.8 , 101309.35], dtype=float32)

In [50]:
p_abs.sel(
    time=slice(1800., 3600.), x=slice(0,2), y=slice(0,2), zn=slice(25., 125.)
    ).data == nc1d.prefn.sel(
        time=slice(1800., 3600.), zn=slice(25., 125.)
        ).data + nc3d.p.sel(
            time=slice(1800., 3600.), x=slice(0,2), y=slice(0,2), zn=slice(25., 125.)
            ).data

array([[[[ True,  True,  True],
         [ True,  True,  True]],

        [[ True,  True,  True],
         [ True,  True,  True]]],


       [[[ True,  True,  True],
         [ True,  True,  True]],

        [[ True,  True,  True],
         [ True,  True,  True]]]])

In [51]:
nc3d['p']

<xarray.DataArray 'p' (time: 2, x: 1280, y: 1280, zn: 121)>
[396492800 values with dtype=float32]
Coordinates:
  * time     (time) float64 1.8e+03 3.6e+03
  * zn       (zn) float64 -25.0 25.0 75.0 ... 5.875e+03 5.925e+03 5.975e+03
Dimensions without coordinates: x, y

In [60]:
ref_p = nc1d.prefn.sel(time=3600., zn=25.).data.tolist()
ref_p

101309.4921875

In [70]:
'dtypes' in dir(nc1d)

False

In [72]:
from numpy import True_, False_

In [74]:
tf = True
tf = True_ if tf else False_
print(tf, type(tf))

True <class 'numpy.bool_'>


In [75]:
'test_data+processed/d20200128_diagnostic_2d_'.strip('_')

'test_data+processed/d20200128_diagnostic_2d'